## Identifying and resolving conflicts in a batch of VI files

ELGs - part 7 is sets 31 to 35

There are 749 visual inspections of a total of 250 unique objects
That gives 2.996 inspections per object.

53 conflicts

In [156]:
save_filename='truth_table_ELGs_sets31_35.txt'

In [2]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

sys.path.append("/global/homes/r/rtojeiro/SV/python")
import VI_utils as VI

#import desispec
# library location will change ..
sys.path.append("/global/homes/r/rtojeiro/prospect/prospect/py")
from prospect import utils_specviewer,plotframes
import matplotlib.pyplot as plt 

In [3]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/'
tiles = ['67230']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 
del obs_db['67230-20200314']
obs_db.keys()

dict_keys(['67230-20200315'])

In [4]:
pd.set_option('display.max_rows', 20)

In [5]:
#set to directory with all the VI files to merge
VI_dir = os.environ['HOME']+'/SV/VI_files/SV0/ELG/'

In [6]:
vi_files = ['desi-vi_SV0_ELG_tile67230_night20200315_31_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_31_SEM.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_31_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_32_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_32_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_32_DRP.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_33_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_33_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_33_BJW.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_34_BJW.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_34_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_34_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_35_HSH.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_35_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_35_ACE.csv']

In [7]:
vi = pd.read_csv(VI_dir + vi_files[0], delimiter = " , ", engine='python')

for i in range(1,len(vi_files)):
    print(vi_files[i])
    vi2 = pd.read_csv(VI_dir + vi_files[i], delimiter = " , ", engine='python')
    vi = vi.append(vi2, ignore_index=True)
    
#make groups of visual inspections, grouped by unique objects, and state number of single and multiple VIs
vi_gp = vi.groupby(['TargetID'])
print('There are ' + str(len(vi)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')
print('That gives ' + str(len(vi)/len(vi_gp)) + ' inspections per object.')

desi-vi_SV0_ELG_tile67230_night20200315_31_SEM.csv
desi-vi_SV0_ELG_tile67230_night20200315_31_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_32_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_32_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_32_DRP.csv
desi-vi_SV0_ELG_tile67230_night20200315_33_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_33_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_33_BJW.csv
desi-vi_SV0_ELG_tile67230_night20200315_34_BJW.csv
desi-vi_SV0_ELG_tile67230_night20200315_34_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_34_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_35_HSH.csv
desi-vi_SV0_ELG_tile67230_night20200315_35_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_35_ACE.csv
There are 749 visual inspections of a total of 250 unique objects
That gives 2.996 inspections per object.


In [8]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment'],
      dtype='object')

### Merge with zbest files
Add: fiberID, delta_chi2, flux information,.. anything else?


In [9]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

tspec_df = tspec['TARGETID','DELTACHI2' ].to_pandas()
tf_df = tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_pandas()

tf_df = tf_df.rename(columns={"TARGETID": "TargetID"})
tspec_df = tspec_df.rename(columns={"TARGETID": "TargetID"})

vi = vi.merge(tf_df, how='left', on='TargetID')
vi = vi.merge(tspec_df, how='left', on='TargetID')

In [10]:
print(vi.keys())
print(len(vi))

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2'],
      dtype='object')
749


### Adding a bunch of useful columns

In [11]:
vi = VI.prep_new_columns(vi)

In [12]:
#check all the new columns (keys) have been added correctly
display(vi)
print(vi.keys())

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
0,35185778187306043,-1,0.33.0.dev3890,0,GALAXY,1.0142,ACE,4,--,--,...,50.728473,1.0142,GALAXY,4.000000,0,0.000000,1.014200,--| OII and balmer lines|--,3,none
1,35185778187306542,-1,0.33.0.dev3890,0,GALAXY,0.9864,ACE,4,--,--,...,679.765608,0.9864,GALAXY,4.000000,0,0.000000,0.986400,MgII in absorption.| OII and balmer lines|--,3,none
2,35185778187306622,-1,0.33.0.dev3890,0,GALAXY,0.6365,ACE,4,--,--,...,710.741031,0.6365,GALAXY,4.000000,0,0.000000,0.636500,--| OII and balmer lines|--,3,none
3,35185778187306634,-1,0.33.0.dev3890,0,GALAXY,1.5635,ACE,2,--,--,...,20.478735,1.5635,GALAXY,1.666667,1,0.000000,1.563500,OII on sky line.|--| Possible [OII] next to s...,3,none
4,35185778187306802,-1,0.33.0.dev3890,0,GALAXY,0.7823,ACE,4,--,--,...,344.300610,0.7823,GALAXY,3.333333,2,0.000000,0.782300,--| OII| Broad [OII] emission,3,none
5,35185778187307161,-1,0.33.0.dev3890,0,GALAXY,0.9542,ACE,4,--,--,...,132.605144,0.9542,GALAXY,4.000000,0,0.000000,0.954200,--| OII doublet and weak balmer lines|--,3,none
6,35185778187307173,-1,0.33.0.dev3890,0,GALAXY,1.1923,ACE,3,--,1.1929,...,431.812797,1.1929,GALAXY,3.333333,1,0.000274,1.192500,--| OII doublet| Triple-peak [OII] emission,3,none
7,35185778187307360,-1,0.33.0.dev3890,0,GALAXY,1.5153,ACE,2,--,1.6162,...,2.014799,1.6162,GALAXY,2.000000,2,0.040114,1.515300,Weak OII| OII possible doublet| Model fits sk...,3,none
8,35185778187307415,-1,0.33.0.dev3890,0,GALAXY,1.3840,ACE,2,--,--,...,8.023933,1.3840,GALAXY,1.333333,2,0.000000,1.384000,Based on OII only| OII ?| Low S/N,3,none
9,35185778187307571,-1,0.33.0.dev3890,0,GALAXY,1.3320,ACE,3,RC,2.7830,...,25.703903,2.7830,QSO,1.666667,3,0.622213,1.332000,Narrow Lyman alpha.| Hb| Not physical model,3,none


Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff', 'dz',
       'vi_combined_z', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')


Get a table that holds only the objects that have been inspected more than once, and for which the individual VI classifications differ by 2 or more, or delta z / (1 + z) > 0.0033, or there is disagreement in best spectype (these are the conflicts to resolve)

In [13]:
vi_gp = vi.groupby(['TargetID'])
vi_conflict = VI.find_conflicts(vi_gp)

Get the target IDs of the problematic objects and display in table form for a quick summary:

In [14]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185778187306802 35185778187307360 35185778187307571 35185778187308731
 35185778187308819 35185778187309518 35185778191500242 35185778191501245
 35185778191501979 35185778191502663 35185778191502699 35185778191503385
 35185778191504286 35185778195693783 35185778195695601 35185778195696327
 35185778195696794 35185778195697232 35185778195697804 35185778195698835
 35185778199888520 35185778199890133 35185778199890294 35185778199890542
 35185778199891596 35185778199892183 35185778204083187 35185778204085120
 35185778204085679 35185778204086026 35185778204086746 35185778204086989
 35185778208279043 35185778208280263 35185778208281832 35185778212471275
 35185778212474885 35185778216665338 35185778216667033 35185778216667241
 35185778216668354 35185778216669670 35185778220861973 35185778220863277
 35185778220864335 35185778220864810 35185778225053791 35185778225053999
 35185778225054325 35185778225054792 35185778225055676 35185778225057501
 35185778229252179]
T

In [15]:
unique_targets[1]

35185778187307360

## This is where I resolve things manually - with care!!
### I think it's better to keep it in a notebook, as typos can be backtracked rather than a single manual edit of a text file

We edit either 'VI class', or 'best redshift' to resolve conflict. At the end, we look for conflicts again and we should find none.


In [16]:
#function to display the conflict in table format and open a prospect window
def display_conflict(conflict_id, show_spectra=None):
    
    #first, remind myself of the problem:
    display(vi[vi.TargetID==unique_targets[conflict_id]][['TargetID', 
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment']])

    # VI interface in notebook
    if show_spectra:
        spectra, zcat= utils_specviewer.load_spectra_zcat_from_targets([unique_targets[conflict_id]], tiledir, obs_db)
        plotframes.plotspectra(spectra, zcatalog=zcat, title='Target_select', notebook=True, mask_type='CMX_TARGET',with_vi_widgets=False)

In [17]:
#first, keep a safe copy of the original dataframe
vi_safe = vi.copy()

We will inspect each conflict on a prospect window, and resolve each conflict in turn

In [18]:
#Keep track of the conflicts by conflict_id

#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
4,35185778187306802,GALAXY,0.7823,ACE,4,--,--,--,--,0.7823,GALAXY,3.333333,2,0.7823,0.0,--| OII| Broad [OII] emission,3,none
53,35185778187306802,GALAXY,0.7823,SEM,2,--,--,--,OII,0.7823,GALAXY,3.333333,2,0.7823,0.0,--| OII| Broad [OII] emission,3,none
103,35185778187306802,GALAXY,0.7823,TWL,4,--,--,--,Broad [OII] emission,0.7823,GALAXY,3.333333,2,0.7823,0.0,--| OII| Broad [OII] emission,3,none


In [19]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
4,35185778187306802,-1,0.33.0.dev3890,0,GALAXY,0.7823,ACE,3.333333,--,--,...,344.30061,0.7823,GALAXY,3.333333,2,0.0,0.7823,--| OII| Broad [OII] emission,3,"flag disagreement, take mean."
53,35185778187306802,-1,0.33.0.dev3890,0,GALAXY,0.7823,SEM,3.333333,--,--,...,344.30061,0.7823,GALAXY,3.333333,2,0.0,0.7823,--| OII| Broad [OII] emission,3,"flag disagreement, take mean."
103,35185778187306802,-1,0.33.0.dev3890,0,GALAXY,0.7823,TWL,3.333333,--,--,...,344.30061,0.7823,GALAXY,3.333333,2,0.0,0.7823,--| OII| Broad [OII] emission,3,"flag disagreement, take mean."


conflict ID =  0


In [20]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
7,35185778187307360,GALAXY,1.5153,ACE,2.0,--,1.6162,GALAXY,Weak OII,1.6162,GALAXY,2.0,2,1.5153,0.040114,Weak OII| OII possible doublet| Model fits sk...,3,none
56,35185778187307360,GALAXY,1.5153,SEM,3.0,--,--,--,OII possible doublet,1.5153,GALAXY,2.0,2,1.5153,0.040114,Weak OII| OII possible doublet| Model fits sk...,3,none
106,35185778187307360,GALAXY,1.5153,TWL,1.0,R,--,--,Model fits sky residuals,1.5153,GALAXY,2.0,2,1.5153,0.040114,Weak OII| OII possible doublet| Model fits sk...,3,none


In [21]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.0
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. z not secure. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
7,35185778187307360,-1,0.33.0.dev3890,0,GALAXY,1.5153,ACE,2.0,--,1.6162,...,2.014799,1.6162,GALAXY,2.0,2,0.040114,1.5153,Weak OII| OII possible doublet| Model fits sk...,3,"flag disagreement, take mean. z not secure. po..."
56,35185778187307360,-1,0.33.0.dev3890,0,GALAXY,1.5153,SEM,2.0,--,--,...,2.014799,1.5153,GALAXY,2.0,2,0.040114,1.5153,Weak OII| OII possible doublet| Model fits sk...,3,"flag disagreement, take mean. z not secure. po..."
106,35185778187307360,-1,0.33.0.dev3890,0,GALAXY,1.5153,TWL,2.0,R,--,...,2.014799,1.5153,GALAXY,2.0,2,0.040114,1.5153,Weak OII| OII possible doublet| Model fits sk...,3,"flag disagreement, take mean. z not secure. po..."


conflict ID =  1


In [22]:
#copy this text to a new cell to display the conflict
conflict_id = 2
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
9,35185778187307571,GALAXY,1.332,ACE,3.0,RC,2.7830,QSO,Narrow Lyman alpha.,2.783,QSO,1.666667,3,1.332,0.622213,Narrow Lyman alpha.| Hb| Not physical model,3,none
58,35185778187307571,GALAXY,1.332,SEM,2.0,--,--,--,Hb,1.332,GALAXY,1.666667,3,1.332,0.622213,Narrow Lyman alpha.| Hb| Not physical model,3,none
108,35185778187307571,GALAXY,1.332,TWL,0.0,--,--,--,Not physical model,1.332,GALAXY,1.666667,3,1.332,0.622213,Narrow Lyman alpha.| Hb| Not physical model,3,none


In [23]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.332
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. spectra too noisy, not secure. take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
9,35185778187307571,-1,0.33.0.dev3890,0,GALAXY,1.332,ACE,1.0,RC,2.7830,...,25.703903,1.332,GALAXY,1.666667,3,0.622213,1.332,Narrow Lyman alpha.| Hb| Not physical model,3,"disagreement in all. spectra too noisy, not se..."
58,35185778187307571,-1,0.33.0.dev3890,0,GALAXY,1.332,SEM,1.0,--,--,...,25.703903,1.332,GALAXY,1.666667,3,0.622213,1.332,Narrow Lyman alpha.| Hb| Not physical model,3,"disagreement in all. spectra too noisy, not se..."
108,35185778187307571,-1,0.33.0.dev3890,0,GALAXY,1.332,TWL,1.0,--,--,...,25.703903,1.332,GALAXY,1.666667,3,0.622213,1.332,Narrow Lyman alpha.| Hb| Not physical model,3,"disagreement in all. spectra too noisy, not se..."


conflict ID =  2


In [24]:
#copy this text to a new cell to display the conflict
conflict_id=3
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
16,35185778187308731,GALAXY,1.253,ACE,4.0,--,--,--,--,1.253,GALAXY,3.0,2,1.253,0.0,--| OII and weak balmer lines| Possible [OII]...,3,none
65,35185778187308731,GALAXY,1.253,SEM,3.0,--,--,--,OII and weak balmer lines,1.253,GALAXY,3.0,2,1.253,0.0,--| OII and weak balmer lines| Possible [OII]...,3,none
115,35185778187308731,GALAXY,1.253,TWL,2.0,--,--,--,Possible [OII] emission next to sky lines; be...,1.253,GALAXY,3.0,2,1.253,0.0,--| OII and weak balmer lines| Possible [OII]...,3,none


In [25]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. OII not secure, no other strong features. flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
16,35185778187308731,-1,0.33.0.dev3890,0,GALAXY,1.253,ACE,2.0,--,--,...,121.075329,1.253,GALAXY,3.0,2,0.0,1.253,--| OII and weak balmer lines| Possible [OII]...,3,"disagreement in flag. OII not secure, no other..."
65,35185778187308731,-1,0.33.0.dev3890,0,GALAXY,1.253,SEM,2.0,--,--,...,121.075329,1.253,GALAXY,3.0,2,0.0,1.253,--| OII and weak balmer lines| Possible [OII]...,3,"disagreement in flag. OII not secure, no other..."
115,35185778187308731,-1,0.33.0.dev3890,0,GALAXY,1.253,TWL,2.0,--,--,...,121.075329,1.253,GALAXY,3.0,2,0.0,1.253,--| OII and weak balmer lines| Possible [OII]...,3,"disagreement in flag. OII not secure, no other..."


conflict ID =  3


In [26]:
#copy this text to a new cell to display the conflict
conflict_id=4
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
17,35185778187308819,GALAXY,0.3491,ACE,3.0,S,0.3484,GALAXY,No data in infrared band,0.3484,GALAXY,1.333333,3,0.348867,0.000519,No data in infrared band| low SNR| Missing pi...,3,none
66,35185778187308819,GALAXY,0.3491,SEM,1.0,--,--,--,low SNR,0.3491,GALAXY,1.333333,3,0.348867,0.000519,No data in infrared band| low SNR| Missing pi...,3,none
116,35185778187308819,GALAXY,0.3491,TWL,0.0,S,--,--,Missing pixels in the red,0.3491,GALAXY,1.333333,3,0.348867,0.000519,No data in infrared band| low SNR| Missing pi...,3,none


In [27]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. missing data from red spectrograph. no strong features. z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
17,35185778187308819,-1,0.33.0.dev3890,0,GALAXY,0.3491,ACE,1.333333,S,0.3484,...,3.350723,0.3484,GALAXY,1.333333,3,0.000519,0.348867,No data in infrared band| low SNR| Missing pi...,3,"flag disagreement, take mean. missing data fro..."
66,35185778187308819,-1,0.33.0.dev3890,0,GALAXY,0.3491,SEM,1.333333,--,--,...,3.350723,0.3491,GALAXY,1.333333,3,0.000519,0.348867,No data in infrared band| low SNR| Missing pi...,3,"flag disagreement, take mean. missing data fro..."
116,35185778187308819,-1,0.33.0.dev3890,0,GALAXY,0.3491,TWL,1.333333,S,--,...,3.350723,0.3491,GALAXY,1.333333,3,0.000519,0.348867,No data in infrared band| low SNR| Missing pi...,3,"flag disagreement, take mean. missing data fro..."


conflict ID =  4


In [28]:
#copy this text to a new cell to display the conflict
conflict_id=5
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
19,35185778187309518,GALAXY,1.3691,ACE,3.0,--,--,--,--,1.3691,GALAXY,3.0,2,1.3691,0.0,--| OII and balmer lines in absorption|Possib...,3,none
68,35185778187309518,GALAXY,1.3691,SEM,4.0,--,--,--,OII and balmer lines in absorption,1.3691,GALAXY,3.0,2,1.3691,0.0,--| OII and balmer lines in absorption|Possib...,3,none
118,35185778187309518,GALAXY,1.3691,TWL,2.0,--,--,--,Possible [OII] between sky lines; being conser...,1.3691,GALAXY,3.0,2,1.3691,0.0,--| OII and balmer lines in absorption|Possib...,3,none


In [29]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. doublet not secure, no other secure features. calibration issues. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)




,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
19,35185778187309518,-1,0.33.0.dev3890,0,GALAXY,1.3691,ACE,2.0,--,--,...,89.332809,1.3691,GALAXY,3.0,2,0.0,1.3691,--| OII and balmer lines in absorption|Possib...,3,"disagreement in flag. doublet not secure, no o..."
68,35185778187309518,-1,0.33.0.dev3890,0,GALAXY,1.3691,SEM,2.0,--,--,...,89.332809,1.3691,GALAXY,3.0,2,0.0,1.3691,--| OII and balmer lines in absorption|Possib...,3,"disagreement in flag. doublet not secure, no o..."
118,35185778187309518,-1,0.33.0.dev3890,0,GALAXY,1.3691,TWL,2.0,--,--,...,89.332809,1.3691,GALAXY,3.0,2,0.0,1.3691,--| OII and balmer lines in absorption|Possib...,3,"disagreement in flag. doublet not secure, no o..."


conflict ID =  5


In [30]:
#copy this text to a new cell to display the conflict
conflict_id=6
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
23,35185778191500242,GALAXY,1.3876,ACE,4.0,R,0.3770,GALAXY,"Narrow Halpha"","" OIII and OII.",0.3770,GALAXY,1.666667,4,1.3876,0.733914,"Narrow Halpha"","" OIII and OII.|--| Model fits...",3,none
72,35185778191500242,GALAXY,1.3876,SEM,1.0,R,--,--,--,1.3876,GALAXY,1.666667,4,1.3876,0.733914,"Narrow Halpha"","" OIII and OII.|--| Model fits...",3,none
122,35185778191500242,GALAXY,1.3876,TWL,0.0,--,--,--,Model fits negative flux,1.3876,GALAXY,1.666667,4,1.3876,0.733914,"Narrow Halpha"","" OIII and OII.|--| Model fits...",3,none


In [31]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.3770
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreememt in flag and z. poor signal. take VI z with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
23,35185778191500242,-1,0.33.0.dev3890,0,GALAXY,1.3876,ACE,2.0,R,0.3770,...,11.757442,0.377,GALAXY,1.666667,4,0.733914,1.3876,"Narrow Halpha"","" OIII and OII.|--| Model fits...",3,disagreememt in flag and z. poor signal. take ...
72,35185778191500242,-1,0.33.0.dev3890,0,GALAXY,1.3876,SEM,2.0,R,--,...,11.757442,0.377,GALAXY,1.666667,4,0.733914,1.3876,"Narrow Halpha"","" OIII and OII.|--| Model fits...",3,disagreememt in flag and z. poor signal. take ...
122,35185778191500242,-1,0.33.0.dev3890,0,GALAXY,1.3876,TWL,2.0,--,--,...,11.757442,0.377,GALAXY,1.666667,4,0.733914,1.3876,"Narrow Halpha"","" OIII and OII.|--| Model fits...",3,disagreememt in flag and z. poor signal. take ...


conflict ID =  6


In [32]:
#copy this text to a new cell to display the conflict
conflict_id=7
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
30,35185778191501245,GALAXY,0.5816,ACE,1.0,--,--,--,No clear match to features.,0.5816,GALAXY,2.0,2,0.5816,0.0,No clear match to features.| OII|Weak [OII] e...,3,none
79,35185778191501245,GALAXY,0.5816,SEM,2.0,--,--,--,OII,0.5816,GALAXY,2.0,2,0.5816,0.0,No clear match to features.| OII|Weak [OII] e...,3,none
129,35185778191501245,GALAXY,0.5816,TWL,3.0,--,--,--,Weak [OII] emission on sky lines,0.5816,GALAXY,2.0,2,0.5816,0.0,No clear match to features.| OII|Weak [OII] e...,3,none


In [33]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.0
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
30,35185778191501245,-1,0.33.0.dev3890,0,GALAXY,0.5816,ACE,2.0,--,--,...,23.069025,0.5816,GALAXY,2.0,2,0.0,0.5816,No clear match to features.| OII|Weak [OII] e...,3,"flag disagreement, take mean.z not secure."
79,35185778191501245,-1,0.33.0.dev3890,0,GALAXY,0.5816,SEM,2.0,--,--,...,23.069025,0.5816,GALAXY,2.0,2,0.0,0.5816,No clear match to features.| OII|Weak [OII] e...,3,"flag disagreement, take mean.z not secure."
129,35185778191501245,-1,0.33.0.dev3890,0,GALAXY,0.5816,TWL,2.0,--,--,...,23.069025,0.5816,GALAXY,2.0,2,0.0,0.5816,No clear match to features.| OII|Weak [OII] e...,3,"flag disagreement, take mean.z not secure."


conflict ID =  7


In [34]:
#copy this text to a new cell to display the conflict
conflict_id=8
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
33,35185778191501979,GALAXY,0.8263,ACE,4.0,C,0.0000,STAR,White dwarf,0.0000,STAR,3.0,3,0.8263,0.8263,White dwarf|--| Point source,3,none
82,35185778191501979,GALAXY,0.8263,SEM,1.0,R,--,--,--,0.8263,GALAXY,3.0,3,0.8263,0.8263,White dwarf|--| Point source,3,none
132,35185778191501979,GALAXY,0.8263,TWL,4.0,C,0.0000,STAR,Point source,0.0000,STAR,3.0,3,0.8263,0.8263,White dwarf|--| Point source,3,none


In [35]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.0
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'STAR'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. 2/3 inspectors VI as a star. secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
33,35185778191501979,-1,0.33.0.dev3890,0,GALAXY,0.8263,ACE,3.0,C,0.0000,...,1.492657,0.0,STAR,3.0,3,0.8263,0.8263,White dwarf|--| Point source,3,disagreement in all. 2/3 inspectors VI as a st...
82,35185778191501979,-1,0.33.0.dev3890,0,GALAXY,0.8263,SEM,3.0,R,--,...,1.492657,0.0,STAR,3.0,3,0.8263,0.8263,White dwarf|--| Point source,3,disagreement in all. 2/3 inspectors VI as a st...
132,35185778191501979,-1,0.33.0.dev3890,0,GALAXY,0.8263,TWL,3.0,C,0.0000,...,1.492657,0.0,STAR,3.0,3,0.8263,0.8263,White dwarf|--| Point source,3,disagreement in all. 2/3 inspectors VI as a st...


conflict ID =  8


In [36]:
#copy this text to a new cell to display the conflict
conflict_id=9
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
35,35185778191502663,GALAXY,1.2553,ACE,4.0,--,--,--,--,1.2553,GALAXY,3.0,2,1.2553,0.0,--| OII| Broad [OII] emission between sky lines,3,none
84,35185778191502663,GALAXY,1.2553,SEM,2.0,--,--,--,OII,1.2553,GALAXY,3.0,2,1.2553,0.0,--| OII| Broad [OII] emission between sky lines,3,none
134,35185778191502663,GALAXY,1.2553,TWL,3.0,--,--,--,Broad [OII] emission between sky lines,1.2553,GALAXY,3.0,2,1.2553,0.0,--| OII| Broad [OII] emission between sky lines,3,none


In [37]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. broadened but unresolved line. no continuum or other secure features. z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
35,35185778191502663,-1,0.33.0.dev3890,0,GALAXY,1.2553,ACE,2.0,--,--,...,244.611487,1.2553,GALAXY,3.0,2,0.0,1.2553,--| OII| Broad [OII] emission between sky lines,3,disagreement in flag. broadened but unresolved...
84,35185778191502663,-1,0.33.0.dev3890,0,GALAXY,1.2553,SEM,2.0,--,--,...,244.611487,1.2553,GALAXY,3.0,2,0.0,1.2553,--| OII| Broad [OII] emission between sky lines,3,disagreement in flag. broadened but unresolved...
134,35185778191502663,-1,0.33.0.dev3890,0,GALAXY,1.2553,TWL,2.0,--,--,...,244.611487,1.2553,GALAXY,3.0,2,0.0,1.2553,--| OII| Broad [OII] emission between sky lines,3,disagreement in flag. broadened but unresolved...


conflict ID =  9


In [38]:
#copy this text to a new cell to display the conflict
conflict_id=10
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
36,35185778191502699,GALAXY,1.4392,ACE,3.0,--,0.6149,GALAXY,OII and OIII.,0.6149,GALAXY,1.333333,3,1.4392,0.510434,OII and OIII.|--| Low S/N and strange model,3,none
85,35185778191502699,GALAXY,1.4392,SEM,1.0,R,1.3733,--,--,1.3733,GALAXY,1.333333,3,1.4392,0.510434,OII and OIII.|--| Low S/N and strange model,3,none
135,35185778191502699,GALAXY,1.4392,TWL,0.0,--,--,--,Low S/N and strange model,1.4392,GALAXY,1.333333,3,1.4392,0.510434,OII and OIII.|--| Low S/N and strange model,3,none


In [39]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.4392
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. too noisy for secure z. take RR answer with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
36,35185778191502699,-1,0.33.0.dev3890,0,GALAXY,1.4392,ACE,1.0,--,0.6149,...,0.692471,1.4392,GALAXY,1.333333,3,0.510434,1.4392,OII and OIII.|--| Low S/N and strange model,3,disagreement in flag and z. too noisy for secu...
85,35185778191502699,-1,0.33.0.dev3890,0,GALAXY,1.4392,SEM,1.0,R,1.3733,...,0.692471,1.4392,GALAXY,1.333333,3,0.510434,1.4392,OII and OIII.|--| Low S/N and strange model,3,disagreement in flag and z. too noisy for secu...
135,35185778191502699,-1,0.33.0.dev3890,0,GALAXY,1.4392,TWL,1.0,--,--,...,0.692471,1.4392,GALAXY,1.333333,3,0.510434,1.4392,OII and OIII.|--| Low S/N and strange model,3,disagreement in flag and z. too noisy for secu...


conflict ID =  10


In [40]:
#copy this text to a new cell to display the conflict
conflict_id=11
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
40,35185778191503385,GALAXY,1.5693,ACE,3.0,--,2.2793,QSO,NV and weak Lyman alpha,2.2793,QSO,2.666667,1,1.5693,0.27634,NV and weak Lyman alpha| OII ?| Possible [OII...,3,none
89,35185778191503385,GALAXY,1.5693,SEM,2.0,--,--,--,OII ?,1.5693,GALAXY,2.666667,1,1.5693,0.27634,NV and weak Lyman alpha| OII ?| Possible [OII...,3,none
139,35185778191503385,GALAXY,1.5693,TWL,3.0,--,--,--,Possible [OII] emission,1.5693,GALAXY,2.666667,1,1.5693,0.27634,NV and weak Lyman alpha| OII ?| Possible [OII...,3,none


In [41]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.5693
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. spectrum too noisy, no secure features, RR answer with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
40,35185778191503385,-1,0.33.0.dev3890,0,GALAXY,1.5693,ACE,2.0,--,2.2793,...,10.10946,1.5693,GALAXY,2.666667,1,0.27634,1.5693,NV and weak Lyman alpha| OII ?| Possible [OII...,3,"disagreement in all. spectrum too noisy, no se..."
89,35185778191503385,-1,0.33.0.dev3890,0,GALAXY,1.5693,SEM,2.0,--,--,...,10.10946,1.5693,GALAXY,2.666667,1,0.27634,1.5693,NV and weak Lyman alpha| OII ?| Possible [OII...,3,"disagreement in all. spectrum too noisy, no se..."
139,35185778191503385,-1,0.33.0.dev3890,0,GALAXY,1.5693,TWL,2.0,--,--,...,10.10946,1.5693,GALAXY,2.666667,1,0.27634,1.5693,NV and weak Lyman alpha| OII ?| Possible [OII...,3,"disagreement in all. spectrum too noisy, no se..."


conflict ID =  11


In [42]:
#copy this text to a new cell to display the conflict
conflict_id=12
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
44,35185778191504286,GALAXY,1.4828,ACE,3.0,R,0.0916,GALAXY,Clear OII,0.0916,GALAXY,1.333333,3,1.4828,1.27446,Clear OII|--| Low S/N; not physical model,3,none
93,35185778191504286,GALAXY,1.4828,SEM,1.0,R,--,--,--,1.4828,GALAXY,1.333333,3,1.4828,1.27446,Clear OII|--| Low S/N; not physical model,3,none
143,35185778191504286,GALAXY,1.4828,TWL,0.0,--,--,--,Low S/N; not physical model,1.4828,GALAXY,1.333333,3,1.4828,1.27446,Clear OII|--| Low S/N; not physical model,3,none


In [43]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.4828
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. no secure features. take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
44,35185778191504286,-1,0.33.0.dev3890,0,GALAXY,1.4828,ACE,1.0,R,0.0916,...,9.670236,1.4828,GALAXY,1.333333,3,1.27446,1.4828,Clear OII|--| Low S/N; not physical model,3,disagreement in flag and z. no secure features...
93,35185778191504286,-1,0.33.0.dev3890,0,GALAXY,1.4828,SEM,1.0,R,--,...,9.670236,1.4828,GALAXY,1.333333,3,1.27446,1.4828,Clear OII|--| Low S/N; not physical model,3,disagreement in flag and z. no secure features...
143,35185778191504286,-1,0.33.0.dev3890,0,GALAXY,1.4828,TWL,1.0,--,--,...,9.670236,1.4828,GALAXY,1.333333,3,1.27446,1.4828,Clear OII|--| Low S/N; not physical model,3,disagreement in flag and z. no secure features...


conflict ID =  12


In [44]:
#copy this text to a new cell to display the conflict
conflict_id=13
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
151,35185778195693783,GALAXY,1.0568,ACE,3.0,--,--,--,--,1.0568,GALAXY,3.0,2,1.0568,0.0,--|--|--,3,none
201,35185778195693783,GALAXY,1.0568,TWL,4.0,--,--,--,--,1.0568,GALAXY,3.0,2,1.0568,0.0,--|--|--,3,none
251,35185778195693783,GALAXY,1.0568,DRP,2.0,--,--,--,--,1.0568,GALAXY,3.0,2,1.0568,0.0,--|--|--,3,none


In [45]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. doublet secures z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
151,35185778195693783,-1,0.33.0.dev3890,0,GALAXY,1.0568,ACE,3.0,--,--,...,76.294451,1.0568,GALAXY,3.0,2,0.0,1.0568,--|--|--,3,"flag disagreement, take mean. doublet secures z."
201,35185778195693783,-1,0.33.0.dev3890,0,GALAXY,1.0568,TWL,3.0,--,--,...,76.294451,1.0568,GALAXY,3.0,2,0.0,1.0568,--|--|--,3,"flag disagreement, take mean. doublet secures z."
251,35185778195693783,-1,0.33.0.dev3890,0,GALAXY,1.0568,DRP,3.0,--,--,...,76.294451,1.0568,GALAXY,3.0,2,0.0,1.0568,--|--|--,3,"flag disagreement, take mean. doublet secures z."


conflict ID =  13


In [46]:
#copy this text to a new cell to display the conflict
conflict_id=14
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
160,35185778195695601,GALAXY,1.4598,ACE,4.0,--,--,--,MgII in absorption.,1.4598,GALAXY,3.333333,2,1.4598,0.0,MgII in absorption.|--|--,3,none
210,35185778195695601,GALAXY,1.4598,TWL,4.0,--,--,--,--,1.4598,GALAXY,3.333333,2,1.4598,0.0,MgII in absorption.|--|--,3,none
260,35185778195695601,GALAXY,1.4598,DRP,2.0,--,--,--,--,1.4598,GALAXY,3.333333,2,1.4598,0.0,MgII in absorption.|--|--,3,none


In [47]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. doublet secures z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
160,35185778195695601,-1,0.33.0.dev3890,0,GALAXY,1.4598,ACE,3.333333,--,--,...,127.972784,1.4598,GALAXY,3.333333,2,0.0,1.4598,MgII in absorption.|--|--,3,"flag disagreement, take mean. doublet secures z."
210,35185778195695601,-1,0.33.0.dev3890,0,GALAXY,1.4598,TWL,3.333333,--,--,...,127.972784,1.4598,GALAXY,3.333333,2,0.0,1.4598,MgII in absorption.|--|--,3,"flag disagreement, take mean. doublet secures z."
260,35185778195695601,-1,0.33.0.dev3890,0,GALAXY,1.4598,DRP,3.333333,--,--,...,127.972784,1.4598,GALAXY,3.333333,2,0.0,1.4598,MgII in absorption.|--|--,3,"flag disagreement, take mean. doublet secures z."


conflict ID =  14


In [48]:
#copy this text to a new cell to display the conflict
conflict_id=15
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
166,35185778195696327,GALAXY,1.2116,ACE,3.0,--,--,--,--,1.2116,GALAXY,3.0,2,1.2116,0.0,--| Triple-peak [OII] emission|--,3,none
216,35185778195696327,GALAXY,1.2116,TWL,4.0,--,--,--,Triple-peak [OII] emission,1.2116,GALAXY,3.0,2,1.2116,0.0,--| Triple-peak [OII] emission|--,3,none
266,35185778195696327,GALAXY,1.2116,DRP,2.0,--,--,--,--,1.2116,GALAXY,3.0,2,1.2116,0.0,--| Triple-peak [OII] emission|--,3,none


In [49]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. broadened but unresolved line, slight continuum change is consistent with OII'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
166,35185778195696327,-1,0.33.0.dev3890,0,GALAXY,1.2116,ACE,3.0,--,--,...,161.799479,1.2116,GALAXY,3.0,2,0.0,1.2116,--| Triple-peak [OII] emission|--,3,"flag disagreement, take mean. broadened but un..."
216,35185778195696327,-1,0.33.0.dev3890,0,GALAXY,1.2116,TWL,3.0,--,--,...,161.799479,1.2116,GALAXY,3.0,2,0.0,1.2116,--| Triple-peak [OII] emission|--,3,"flag disagreement, take mean. broadened but un..."
266,35185778195696327,-1,0.33.0.dev3890,0,GALAXY,1.2116,DRP,3.0,--,--,...,161.799479,1.2116,GALAXY,3.0,2,0.0,1.2116,--| Triple-peak [OII] emission|--,3,"flag disagreement, take mean. broadened but un..."


conflict ID =  15


In [50]:
#copy this text to a new cell to display the conflict
conflict_id=16
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
168,35185778195696794,GALAXY,1.4129,ACE,3.0,--,0.8891,GALAXY,Based on OII and Hbeta only,0.8891,GALAXY,1.0,3,1.4129,0.277275,Based on OII and Hbeta only| Low S/N|--,3,none
218,35185778195696794,GALAXY,1.4129,TWL,0.0,--,--,--,Low S/N,1.4129,GALAXY,1.0,3,1.4129,0.277275,Based on OII and Hbeta only| Low S/N|--,3,none
268,35185778195696794,GALAXY,1.4129,DRP,0.0,--,--,--,--,1.4129,GALAXY,1.0,3,1.4129,0.277275,Based on OII and Hbeta only| Low S/N|--,3,none


In [51]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.8891
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. noisy. possible VI z based on weak OII. take with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
168,35185778195696794,-1,0.33.0.dev3890,0,GALAXY,1.4129,ACE,2.0,--,0.8891,...,22.434322,0.8891,GALAXY,1.0,3,0.277275,1.4129,Based on OII and Hbeta only| Low S/N|--,3,disagreement in z and flag. noisy. possible VI...
218,35185778195696794,-1,0.33.0.dev3890,0,GALAXY,1.4129,TWL,2.0,--,--,...,22.434322,0.8891,GALAXY,1.0,3,0.277275,1.4129,Based on OII and Hbeta only| Low S/N|--,3,disagreement in z and flag. noisy. possible VI...
268,35185778195696794,-1,0.33.0.dev3890,0,GALAXY,1.4129,DRP,2.0,--,--,...,22.434322,0.8891,GALAXY,1.0,3,0.277275,1.4129,Based on OII and Hbeta only| Low S/N|--,3,disagreement in z and flag. noisy. possible VI...


conflict ID =  16


In [52]:
#copy this text to a new cell to display the conflict
conflict_id=17
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
170,35185778195697232,GALAXY,1.3841,ACE,3.0,RC,2.8330,QSO,Based on Lyman alpha,2.8330,QSO,1.0,3,1.3841,0.607735,Based on Lyman alpha| Sky over subtraction; L...,3,none
220,35185778195697232,GALAXY,1.3841,TWL,0.0,S,--,--,Sky over subtraction; Low S/N,1.3841,GALAXY,1.0,3,1.3841,0.607735,Based on Lyman alpha| Sky over subtraction; L...,3,none
270,35185778195697232,GALAXY,1.3841,DRP,0.0,--,--,--,--,1.3841,GALAXY,1.0,3,1.3841,0.607735,Based on Lyman alpha| Sky over subtraction; L...,3,none


In [53]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 2.8330
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, z and spectype. very noisy. 1/3 inspectors suggests QSO VI z based on LyA. not very convincing line, take with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
170,35185778195697232,-1,0.33.0.dev3890,0,GALAXY,1.3841,ACE,2.0,RC,2.8330,...,5.10057,2.833,QSO,1.0,3,0.607735,1.3841,Based on Lyman alpha| Sky over subtraction; L...,3,"disagreement in flag, z and spectype. very noi..."
220,35185778195697232,-1,0.33.0.dev3890,0,GALAXY,1.3841,TWL,2.0,S,--,...,5.10057,2.833,QSO,1.0,3,0.607735,1.3841,Based on Lyman alpha| Sky over subtraction; L...,3,"disagreement in flag, z and spectype. very noi..."
270,35185778195697232,-1,0.33.0.dev3890,0,GALAXY,1.3841,DRP,2.0,--,--,...,5.10057,2.833,QSO,1.0,3,0.607735,1.3841,Based on Lyman alpha| Sky over subtraction; L...,3,"disagreement in flag, z and spectype. very noi..."


conflict ID =  17


In [54]:
#copy this text to a new cell to display the conflict
conflict_id=18
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
178,35185778195697804,GALAXY,0.7557,ACE,3.0,--,0.8182,GALAXY,Based on OII and Hbeta,0.8182,GALAXY,3.0,2,0.7557,0.035598,Based on OII and Hbeta|--|--,3,none
228,35185778195697804,GALAXY,0.7557,TWL,4.0,--,--,--,--,0.7557,GALAXY,3.0,2,0.7557,0.035598,Based on OII and Hbeta|--|--,3,none
278,35185778195697804,GALAXY,0.7557,DRP,2.0,--,--,--,--,0.7557,GALAXY,3.0,2,0.7557,0.035598,Based on OII and Hbeta|--|--,3,none


In [55]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.8182
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. unconvincing fit, misses an emission line. take 1 VI z, flag 2. '

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
178,35185778195697804,-1,0.33.0.dev3890,0,GALAXY,0.7557,ACE,2.0,--,0.8182,...,24.446693,0.8182,GALAXY,3.0,2,0.035598,0.7557,Based on OII and Hbeta|--|--,3,"disagreement in flag and z. unconvincing fit, ..."
228,35185778195697804,-1,0.33.0.dev3890,0,GALAXY,0.7557,TWL,2.0,--,--,...,24.446693,0.8182,GALAXY,3.0,2,0.035598,0.7557,Based on OII and Hbeta|--|--,3,"disagreement in flag and z. unconvincing fit, ..."
278,35185778195697804,-1,0.33.0.dev3890,0,GALAXY,0.7557,DRP,2.0,--,--,...,24.446693,0.8182,GALAXY,3.0,2,0.035598,0.7557,Based on OII and Hbeta|--|--,3,"disagreement in flag and z. unconvincing fit, ..."


conflict ID =  18


In [56]:
#copy this text to a new cell to display the conflict
conflict_id=19
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
173,35185778195698835,GALAXY,1.2083,ACE,4.0,--,--,--,--,1.2083,GALAXY,3.333333,2,1.2083,0.0,--|--|--,3,none
223,35185778195698835,GALAXY,1.2083,TWL,4.0,--,--,--,--,1.2083,GALAXY,3.333333,2,1.2083,0.0,--|--|--,3,none
273,35185778195698835,GALAXY,1.2083,DRP,2.0,--,--,--,--,1.2083,GALAXY,3.333333,2,1.2083,0.0,--|--|--,3,none


In [57]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. doublet secures z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
173,35185778195698835,-1,0.33.0.dev3890,0,GALAXY,1.2083,ACE,3.333333,--,--,...,568.319482,1.2083,GALAXY,3.333333,2,0.0,1.2083,--|--|--,3,"flag disagreement, take mean. doublet secures z."
223,35185778195698835,-1,0.33.0.dev3890,0,GALAXY,1.2083,TWL,3.333333,--,--,...,568.319482,1.2083,GALAXY,3.333333,2,0.0,1.2083,--|--|--,3,"flag disagreement, take mean. doublet secures z."
273,35185778195698835,-1,0.33.0.dev3890,0,GALAXY,1.2083,DRP,3.333333,--,--,...,568.319482,1.2083,GALAXY,3.333333,2,0.0,1.2083,--|--|--,3,"flag disagreement, take mean. doublet secures z."


conflict ID =  19


In [58]:
#copy this text to a new cell to display the conflict
conflict_id=20
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
180,35185778199888520,GALAXY,0.6798,ACE,3.0,R,1.2566,GALAXY,"Continuum """"hump"""" 6800-7600A. Redrock confus...",1.2566,GALAXY,3.0,0,0.6798,0.343374,"Continuum """"hump"""" 6800-7600A. Redrock confus...",3,none
230,35185778199888520,GALAXY,0.6798,TWL,3.0,R,1.2565,--,Possible [OII] at z~1.2565; on top of a sky line,1.2565,GALAXY,3.0,0,0.6798,0.343374,"Continuum """"hump"""" 6800-7600A. Redrock confus...",3,none
280,35185778199888520,GALAXY,0.6798,DRP,3.0,--,--,--,--,0.6798,GALAXY,3.0,0,0.6798,0.343374,"Continuum """"hump"""" 6800-7600A. Redrock confus...",3,none


In [59]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2565
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. 2/3 inspectors give VI z based on OII. take with their mean flag of 3.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
180,35185778199888520,-1,0.33.0.dev3890,0,GALAXY,0.6798,ACE,3.0,R,1.2566,...,21.096912,1.2565,GALAXY,3.0,0,0.343374,0.6798,"Continuum """"hump"""" 6800-7600A. Redrock confus...",3,disagreement in z and flag. 2/3 inspectors giv...
230,35185778199888520,-1,0.33.0.dev3890,0,GALAXY,0.6798,TWL,3.0,R,1.2565,...,21.096912,1.2565,GALAXY,3.0,0,0.343374,0.6798,"Continuum """"hump"""" 6800-7600A. Redrock confus...",3,disagreement in z and flag. 2/3 inspectors giv...
280,35185778199888520,-1,0.33.0.dev3890,0,GALAXY,0.6798,DRP,3.0,--,--,...,21.096912,1.2565,GALAXY,3.0,0,0.343374,0.6798,"Continuum """"hump"""" 6800-7600A. Redrock confus...",3,disagreement in z and flag. 2/3 inspectors giv...


conflict ID =  20


In [60]:
#copy this text to a new cell to display the conflict
conflict_id=21
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
184,35185778199890133,QSO,4.553,ACE,4.0,RC,0.8541,GALAXY,"Continuum """"hump"""" 6800-7600A.",0.8541,GALAXY,2.666667,4,4.553,1.995307,"Continuum """"hump"""" 6800-7600A.|Triple-peak [O...",3,none
234,35185778199890133,QSO,4.553,TWL,4.0,C,0.8539,GALAXY,Triple-peak [OII] at z~0.8539; Calibration fea...,0.8539,GALAXY,2.666667,4,4.553,1.995307,"Continuum """"hump"""" 6800-7600A.|Triple-peak [O...",3,none
284,35185778199890133,QSO,4.553,DRP,0.0,--,--,--,--,4.5530,QSO,2.666667,4,4.553,1.995307,"Continuum """"hump"""" 6800-7600A.|Triple-peak [O...",3,none


In [61]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.854
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. 2/3 inspectors give same VI based on strong emission line with a strange profile - consistent with triple peak OII. take with their mean flag. unreliable continuum, calibration issues. not a QSO.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
184,35185778199890133,-1,0.33.0.dev3890,0,QSO,4.553,ACE,4.0,RC,0.8541,...,32.077388,0.854,GALAXY,2.666667,4,1.995307,4.553,"Continuum """"hump"""" 6800-7600A.|Triple-peak [O...",3,disagreement in all. 2/3 inspectors give same ...
234,35185778199890133,-1,0.33.0.dev3890,0,QSO,4.553,TWL,4.0,C,0.8539,...,32.077388,0.854,GALAXY,2.666667,4,1.995307,4.553,"Continuum """"hump"""" 6800-7600A.|Triple-peak [O...",3,disagreement in all. 2/3 inspectors give same ...
284,35185778199890133,-1,0.33.0.dev3890,0,QSO,4.553,DRP,4.0,--,--,...,32.077388,0.854,GALAXY,2.666667,4,1.995307,4.553,"Continuum """"hump"""" 6800-7600A.|Triple-peak [O...",3,disagreement in all. 2/3 inspectors give same ...


conflict ID =  21


In [62]:
#copy this text to a new cell to display the conflict
conflict_id=22
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
194,35185778199890294,GALAXY,0.7192,ACE,4.0,--,--,--,--,0.7192,GALAXY,2.666667,3,0.7192,0.0,--| Low S/N|--,3,none
244,35185778199890294,GALAXY,0.7192,TWL,1.0,--,--,--,Low S/N,0.7192,GALAXY,2.666667,3,0.7192,0.0,--| Low S/N|--,3,none
294,35185778199890294,GALAXY,0.7192,DRP,3.0,--,--,--,--,0.7192,GALAXY,2.666667,3,0.7192,0.0,--| Low S/N|--,3,none


In [63]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.666667
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. weak OII doublet.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
194,35185778199890294,-1,0.33.0.dev3890,0,GALAXY,0.7192,ACE,2.666667,--,--,...,19.712183,0.7192,GALAXY,2.666667,3,0.0,0.7192,--| Low S/N|--,3,"flag disagreement, take mean. weak OII doublet."
244,35185778199890294,-1,0.33.0.dev3890,0,GALAXY,0.7192,TWL,2.666667,--,--,...,19.712183,0.7192,GALAXY,2.666667,3,0.0,0.7192,--| Low S/N|--,3,"flag disagreement, take mean. weak OII doublet."
294,35185778199890294,-1,0.33.0.dev3890,0,GALAXY,0.7192,DRP,2.666667,--,--,...,19.712183,0.7192,GALAXY,2.666667,3,0.0,0.7192,--| Low S/N|--,3,"flag disagreement, take mean. weak OII doublet."


conflict ID =  22


In [64]:
#copy this text to a new cell to display the conflict
conflict_id=23
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
186,35185778199890542,GALAXY,0.6263,ACE,4.0,R,1.1844,GALAXY,"Continuum """"hump"""" 6800-7600A.",1.1844,GALAXY,3.333333,1,0.6263,0.343233,"Continuum """"hump"""" 6800-7600A.| [OII] at z~1....",3,none
236,35185778199890542,GALAXY,0.6263,TWL,3.0,RS,1.1845,--,[OII] at z~1.1845; Calibration features aroun...,1.1845,GALAXY,3.333333,1,0.6263,0.343233,"Continuum """"hump"""" 6800-7600A.| [OII] at z~1....",3,none
286,35185778199890542,GALAXY,0.6263,DRP,3.0,--,--,--,--,0.6263,GALAXY,3.333333,1,0.6263,0.343233,"Continuum """"hump"""" 6800-7600A.| [OII] at z~1....",3,none


In [65]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.1845
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 2/3 inspectors suggest VI z based on OII. take with their mean flag. calibration issues. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
186,35185778199890542,-1,0.33.0.dev3890,0,GALAXY,0.6263,ACE,3.5,R,1.1844,...,148.296152,1.1845,GALAXY,3.333333,1,0.343233,0.6263,"Continuum """"hump"""" 6800-7600A.| [OII] at z~1....",3,disagreement in flag and z. 2/3 inspectors sug...
236,35185778199890542,-1,0.33.0.dev3890,0,GALAXY,0.6263,TWL,3.5,RS,1.1845,...,148.296152,1.1845,GALAXY,3.333333,1,0.343233,0.6263,"Continuum """"hump"""" 6800-7600A.| [OII] at z~1....",3,disagreement in flag and z. 2/3 inspectors sug...
286,35185778199890542,-1,0.33.0.dev3890,0,GALAXY,0.6263,DRP,3.5,--,--,...,148.296152,1.1845,GALAXY,3.333333,1,0.343233,0.6263,"Continuum """"hump"""" 6800-7600A.| [OII] at z~1....",3,disagreement in flag and z. 2/3 inspectors sug...


conflict ID =  23


In [66]:
#copy this text to a new cell to display the conflict
conflict_id=24
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
188,35185778199891596,QSO,4.5468,ACE,4.0,RC,--,--,"Continuum """"hump"""" 6800-7600A.",4.5468,QSO,2.666667,4,4.5468,1.946664,"Continuum """"hump"""" 6800-7600A.| Triple peak [...",3,none
238,35185778199891596,QSO,4.5468,TWL,4.0,RS,0.8824,--,Triple peak [OII]; Calibration features ~6800 A,0.8824,QSO,2.666667,4,4.5468,1.946664,"Continuum """"hump"""" 6800-7600A.| Triple peak [...",3,none
288,35185778199891596,QSO,4.5468,DRP,0.0,--,--,--,--,4.5468,QSO,2.666667,4,4.5468,1.946664,"Continuum """"hump"""" 6800-7600A.| Triple peak [...",3,none


In [67]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.8824
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. 1/3 inspectors gives VI z based on OII missed by RR. odd continuum break in green arm, calibration issues. not a QSO'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
188,35185778199891596,-1,0.33.0.dev3890,0,QSO,4.5468,ACE,3.0,RC,--,...,4.407445,0.8824,GALAXY,2.666667,4,1.946664,4.5468,"Continuum """"hump"""" 6800-7600A.| Triple peak [...",3,disagreement in z and flag. 1/3 inspectors giv...
238,35185778199891596,-1,0.33.0.dev3890,0,QSO,4.5468,TWL,3.0,RS,0.8824,...,4.407445,0.8824,GALAXY,2.666667,4,1.946664,4.5468,"Continuum """"hump"""" 6800-7600A.| Triple peak [...",3,disagreement in z and flag. 1/3 inspectors giv...
288,35185778199891596,-1,0.33.0.dev3890,0,QSO,4.5468,DRP,3.0,--,--,...,4.407445,0.8824,GALAXY,2.666667,4,1.946664,4.5468,"Continuum """"hump"""" 6800-7600A.| Triple peak [...",3,disagreement in z and flag. 1/3 inspectors giv...


conflict ID =  24


In [68]:
#copy this text to a new cell to display the conflict
conflict_id=25
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
190,35185778199892183,GALAXY,0.507,ACE,-1.0,--,1.0240,GALAXY,"Continuum """"hump"""" 6800-7600A.Redrock confuse...",1.024,GALAXY,1.666667,5,0.507,0.343066,"Continuum """"hump"""" 6800-7600A.Redrock confuse...",3,none
240,35185778199892183,GALAXY,0.507,TWL,4.0,RS,1.0240,--,Calibration features around 6800 A,1.024,GALAXY,1.666667,5,0.507,0.343066,"Continuum """"hump"""" 6800-7600A.Redrock confuse...",3,none
290,35185778199892183,GALAXY,0.507,DRP,2.0,--,--,--,--,0.507,GALAXY,1.666667,5,0.507,0.343066,"Continuum """"hump"""" 6800-7600A.Redrock confuse...",3,none


In [69]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0240
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 2/3 inspectors give same VI z based on doublet missed by RR. odd continuum break in green arm.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
190,35185778199892183,-1,0.33.0.dev3890,0,GALAXY,0.507,ACE,3.0,--,1.0240,...,385.976197,1.024,GALAXY,1.666667,5,0.343066,0.507,"Continuum """"hump"""" 6800-7600A.Redrock confuse...",3,disagreement in flag and z. 2/3 inspectors giv...
240,35185778199892183,-1,0.33.0.dev3890,0,GALAXY,0.507,TWL,3.0,RS,1.0240,...,385.976197,1.024,GALAXY,1.666667,5,0.343066,0.507,"Continuum """"hump"""" 6800-7600A.Redrock confuse...",3,disagreement in flag and z. 2/3 inspectors giv...
290,35185778199892183,-1,0.33.0.dev3890,0,GALAXY,0.507,DRP,3.0,--,--,...,385.976197,1.024,GALAXY,1.666667,5,0.343066,0.507,"Continuum """"hump"""" 6800-7600A.Redrock confuse...",3,disagreement in flag and z. 2/3 inspectors giv...


conflict ID =  25


In [70]:
#copy this text to a new cell to display the conflict
conflict_id=26
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
304,35185778204083187,GALAXY,1.0916,ACE,4.0,--,1.0921,GALAXY,--,1.0921,GALAXY,2.666667,2,1.091967,0.000287,--| Possible [OII] emission; being conservativ...,3,none
354,35185778204083187,GALAXY,1.0916,TWL,2.0,--,1.0922,--,Possible [OII] emission; being conservative,1.0922,GALAXY,2.666667,2,1.091967,0.000287,--| Possible [OII] emission; being conservativ...,3,none
404,35185778204083187,GALAXY,1.0916,BJW,2.0,--,--,--,low S/N; questionable [O II] in sky lines,1.0916,GALAXY,2.666667,2,1.091967,0.000287,--| Possible [OII] emission; being conservativ...,3,none


In [71]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0921
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. OII not secure, flag 2. shift z slighty to better match profile.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
304,35185778204083187,-1,0.33.0.dev3890,0,GALAXY,1.0916,ACE,2.0,--,1.0921,...,58.636263,1.0921,GALAXY,2.666667,2,0.000287,1.091967,--| Possible [OII] emission; being conservativ...,3,"disagreement in flag. OII not secure, flag 2. ..."
354,35185778204083187,-1,0.33.0.dev3890,0,GALAXY,1.0916,TWL,2.0,--,1.0922,...,58.636263,1.0921,GALAXY,2.666667,2,0.000287,1.091967,--| Possible [OII] emission; being conservativ...,3,"disagreement in flag. OII not secure, flag 2. ..."
404,35185778204083187,-1,0.33.0.dev3890,0,GALAXY,1.0916,BJW,2.0,--,--,...,58.636263,1.0921,GALAXY,2.666667,2,0.000287,1.091967,--| Possible [OII] emission; being conservativ...,3,"disagreement in flag. OII not secure, flag 2. ..."


conflict ID =  26


In [72]:
#copy this text to a new cell to display the conflict
conflict_id=27
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
307,35185778204085120,GALAXY,1.5681,ACE,3.0,--,--,--,--,1.5681,GALAXY,2.0,2,1.5681,0.0,--|Possible [OII] next to sky lines| [O II] on...,3,none
357,35185778204085120,GALAXY,1.5681,TWL,2.0,--,--,--,Possible [OII] next to sky lines,1.5681,GALAXY,2.0,2,1.5681,0.0,--|Possible [OII] next to sky lines| [O II] on...,3,none
407,35185778204085120,GALAXY,1.5681,BJW,1.0,--,--,--,[O II] on a sky line; likely not real,1.5681,GALAXY,2.0,2,1.5681,0.0,--|Possible [OII] next to sky lines| [O II] on...,3,none


In [73]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
307,35185778204085120,-1,0.33.0.dev3890,0,GALAXY,1.5681,ACE,2.0,--,--,...,5.708787,1.5681,GALAXY,2.0,2,0.0,1.5681,--|Possible [OII] next to sky lines| [O II] on...,3,"flag disagreement, take mean. z not secure."
357,35185778204085120,-1,0.33.0.dev3890,0,GALAXY,1.5681,TWL,2.0,--,--,...,5.708787,1.5681,GALAXY,2.0,2,0.0,1.5681,--|Possible [OII] next to sky lines| [O II] on...,3,"flag disagreement, take mean. z not secure."
407,35185778204085120,-1,0.33.0.dev3890,0,GALAXY,1.5681,BJW,2.0,--,--,...,5.708787,1.5681,GALAXY,2.0,2,0.0,1.5681,--|Possible [OII] next to sky lines| [O II] on...,3,"flag disagreement, take mean. z not secure."


conflict ID =  27


In [74]:
#copy this text to a new cell to display the conflict
conflict_id=28
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
310,35185778204085679,GALAXY,0.1719,ACE,3.0,--,1.0727,GALAXY,--,1.0727,GALAXY,1.666667,3,0.1719,0.768666,--| Low S/N; hard to tell whether the best fit...,3,none
360,35185778204085679,GALAXY,0.1719,TWL,2.0,--,--,--,Low S/N; hard to tell whether the best fit mo...,0.1719,GALAXY,1.666667,3,0.1719,0.768666,--| Low S/N; hard to tell whether the best fit...,3,none
410,35185778204085679,GALAXY,0.1719,BJW,0.0,R,--,--,just fitting noise,0.1719,GALAXY,1.666667,3,0.1719,0.768666,--| Low S/N; hard to tell whether the best fit...,3,none


In [75]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.666667
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
310,35185778204085679,-1,0.33.0.dev3890,0,GALAXY,0.1719,ACE,1.666667,--,1.0727,...,7.257994,1.0727,GALAXY,1.666667,3,0.768666,0.1719,--| Low S/N; hard to tell whether the best fit...,3,"flag disagreement, take mean. z not secure."
360,35185778204085679,-1,0.33.0.dev3890,0,GALAXY,0.1719,TWL,1.666667,--,--,...,7.257994,0.1719,GALAXY,1.666667,3,0.768666,0.1719,--| Low S/N; hard to tell whether the best fit...,3,"flag disagreement, take mean. z not secure."
410,35185778204085679,-1,0.33.0.dev3890,0,GALAXY,0.1719,BJW,1.666667,R,--,...,7.257994,0.1719,GALAXY,1.666667,3,0.768666,0.1719,--| Low S/N; hard to tell whether the best fit...,3,"flag disagreement, take mean. z not secure."


conflict ID =  28


In [76]:
#copy this text to a new cell to display the conflict
conflict_id=29
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
301,35185778204086026,GALAXY,1.1023,ACE,4.0,--,--,--,--,1.1023,GALAXY,3.0,2,1.1023,0.0,--| Possible [OII] emission with strange profi...,3,none
351,35185778204086026,GALAXY,1.1023,TWL,2.0,--,--,--,Possible [OII] emission with strange profile,1.1023,GALAXY,3.0,2,1.1023,0.0,--| Possible [OII] emission with strange profi...,3,none
401,35185778204086026,GALAXY,1.1023,BJW,3.0,--,--,--,mediocre S/N but asymmetric [O II] likely real,1.1023,GALAXY,3.0,2,1.1023,0.0,--| Possible [OII] emission with strange profi...,3,none


In [77]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
301,35185778204086026,-1,0.33.0.dev3890,0,GALAXY,1.1023,ACE,3.0,--,--,...,66.448461,1.1023,GALAXY,3.0,2,0.0,1.1023,--| Possible [OII] emission with strange profi...,3,"flag disagreement, take mean. z is secure."
351,35185778204086026,-1,0.33.0.dev3890,0,GALAXY,1.1023,TWL,3.0,--,--,...,66.448461,1.1023,GALAXY,3.0,2,0.0,1.1023,--| Possible [OII] emission with strange profi...,3,"flag disagreement, take mean. z is secure."
401,35185778204086026,-1,0.33.0.dev3890,0,GALAXY,1.1023,BJW,3.0,--,--,...,66.448461,1.1023,GALAXY,3.0,2,0.0,1.1023,--| Possible [OII] emission with strange profi...,3,"flag disagreement, take mean. z is secure."


conflict ID =  29


In [78]:
#copy this text to a new cell to display the conflict
conflict_id=30
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
314,35185778204086746,GALAXY,1.0593,ACE,4.0,--,0.4559,GALAXY,Hdelta to Halpha but no OII,0.4559,GALAXY,1.333333,4,1.0593,0.414452,Hdelta to Halpha but no OII|Unphysical model|...,3,none
364,35185778204086746,GALAXY,1.0593,TWL,0.0,--,--,--,Unphysical model,1.0593,GALAXY,1.333333,4,1.0593,0.414452,Hdelta to Halpha but no OII|Unphysical model|...,3,none
414,35185778204086746,GALAXY,1.0593,BJW,0.0,R,--,--,no features,1.0593,GALAXY,1.333333,4,1.0593,0.414452,Hdelta to Halpha but no OII|Unphysical model|...,3,none


In [79]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0593
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. too noisy for secure z. take RR answer with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
314,35185778204086746,-1,0.33.0.dev3890,0,GALAXY,1.0593,ACE,1.0,--,0.4559,...,2.272552,1.0593,GALAXY,1.333333,4,0.414452,1.0593,Hdelta to Halpha but no OII|Unphysical model|...,3,disagreement in z and flag. too noisy for secu...
364,35185778204086746,-1,0.33.0.dev3890,0,GALAXY,1.0593,TWL,1.0,--,--,...,2.272552,1.0593,GALAXY,1.333333,4,0.414452,1.0593,Hdelta to Halpha but no OII|Unphysical model|...,3,disagreement in z and flag. too noisy for secu...
414,35185778204086746,-1,0.33.0.dev3890,0,GALAXY,1.0593,BJW,1.0,R,--,...,2.272552,1.0593,GALAXY,1.333333,4,0.414452,1.0593,Hdelta to Halpha but no OII|Unphysical model|...,3,disagreement in z and flag. too noisy for secu...


conflict ID =  30


In [80]:
#copy this text to a new cell to display the conflict
conflict_id=31
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
303,35185778204086989,GALAXY,0.9668,ACE,4.0,--,--,--,--,0.9668,GALAXY,3.333333,2,0.967033,0.000356,--| Calibration features| bad continuum; [O II...,3,none
353,35185778204086989,GALAXY,0.9668,TWL,4.0,--,0.9675,--,Calibration features,0.9675,GALAXY,3.333333,2,0.967033,0.000356,--| Calibration features| bad continuum; [O II...,3,none
403,35185778204086989,GALAXY,0.9668,BJW,2.0,S,--,--,bad continuum; [O II] might be real,0.9668,GALAXY,3.333333,2,0.967033,0.000356,--| Calibration features| bad continuum; [O II...,3,none


In [81]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. z not secure. single line, not resolved. unphysical continuum, calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
303,35185778204086989,-1,0.33.0.dev3890,0,GALAXY,0.9668,ACE,2.0,--,--,...,7.129699,0.9668,GALAXY,3.333333,2,0.000356,0.967033,--| Calibration features| bad continuum; [O II...,3,disagreement in flag. z not secure. single lin...
353,35185778204086989,-1,0.33.0.dev3890,0,GALAXY,0.9668,TWL,2.0,--,0.9675,...,7.129699,0.9675,GALAXY,3.333333,2,0.000356,0.967033,--| Calibration features| bad continuum; [O II...,3,disagreement in flag. z not secure. single lin...
403,35185778204086989,-1,0.33.0.dev3890,0,GALAXY,0.9668,BJW,2.0,S,--,...,7.129699,0.9668,GALAXY,3.333333,2,0.000356,0.967033,--| Calibration features| bad continuum; [O II...,3,disagreement in flag. z not secure. single lin...


conflict ID =  31


In [82]:
#copy this text to a new cell to display the conflict
conflict_id=32
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
336,35185778208279043,GALAXY,0.9794,ACE,2.0,RC,0.0000,STAR,--,0.0000,STAR,2.0,4,0.9794,0.9794,--| Point source| no features,3,none
386,35185778208279043,GALAXY,0.9794,TWL,4.0,C,0.0000,STAR,Point source,0.0000,STAR,2.0,4,0.9794,0.9794,--| Point source| no features,3,none
436,35185778208279043,GALAXY,0.9794,BJW,0.0,R,--,--,no features,0.9794,GALAXY,2.0,4,0.9794,0.9794,--| Point source| no features,3,none


In [83]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'STAR'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. 2/3 inspectors say STAR. take with flag 3.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
336,35185778208279043,-1,0.33.0.dev3890,0,GALAXY,0.9794,ACE,3.0,RC,0.0000,...,1.214865,0.0,STAR,2.0,4,0.9794,0.9794,--| Point source| no features,3,disagreement in all. 2/3 inspectors say STAR. ...
386,35185778208279043,-1,0.33.0.dev3890,0,GALAXY,0.9794,TWL,3.0,C,0.0000,...,1.214865,0.0,STAR,2.0,4,0.9794,0.9794,--| Point source| no features,3,disagreement in all. 2/3 inspectors say STAR. ...
436,35185778208279043,-1,0.33.0.dev3890,0,GALAXY,0.9794,BJW,3.0,R,--,...,1.214865,0.0,STAR,2.0,4,0.9794,0.9794,--| Point source| no features,3,disagreement in all. 2/3 inspectors say STAR. ...


conflict ID =  32


In [84]:
#copy this text to a new cell to display the conflict
conflict_id=33
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
340,35185778208280263,GALAXY,0.5083,ACE,3.0,--,--,--,--,0.5083,GALAXY,1.666667,3,0.5083,0.0,--| Possible [OIII] emission| no real features,3,none
390,35185778208280263,GALAXY,0.5083,TWL,2.0,--,--,--,Possible [OIII] emission,0.5083,GALAXY,1.666667,3,0.5083,0.0,--| Possible [OIII] emission| no real features,3,none
440,35185778208280263,GALAXY,0.5083,BJW,0.0,R,--,--,no real features,0.5083,GALAXY,1.666667,3,0.5083,0.0,--| Possible [OIII] emission| no real features,3,none


In [85]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.666667
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
340,35185778208280263,-1,0.33.0.dev3890,0,GALAXY,0.5083,ACE,1.666667,--,--,...,8.403651,0.5083,GALAXY,1.666667,3,0.0,0.5083,--| Possible [OIII] emission| no real features,3,"flag disagreement, take mean. z not secure."
390,35185778208280263,-1,0.33.0.dev3890,0,GALAXY,0.5083,TWL,1.666667,--,--,...,8.403651,0.5083,GALAXY,1.666667,3,0.0,0.5083,--| Possible [OIII] emission| no real features,3,"flag disagreement, take mean. z not secure."
440,35185778208280263,-1,0.33.0.dev3890,0,GALAXY,0.5083,BJW,1.666667,R,--,...,8.403651,0.5083,GALAXY,1.666667,3,0.0,0.5083,--| Possible [OIII] emission| no real features,3,"flag disagreement, take mean. z not secure."


conflict ID =  33


In [86]:
#copy this text to a new cell to display the conflict
conflict_id=34
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
345,35185778208281832,GALAXY,1.6243,ACE,3.0,--,1.6230,GALAXY,--,1.6230,GALAXY,2.0,3,1.6234,0.000534,--|Possible [OII] next to sky lines| no features,3,none
395,35185778208281832,GALAXY,1.6243,TWL,3.0,--,1.6229,--,Possible [OII] next to sky lines,1.6229,GALAXY,2.0,3,1.6234,0.000534,--|Possible [OII] next to sky lines| no features,3,none
445,35185778208281832,GALAXY,1.6243,BJW,0.0,R,--,--,no features,1.6243,GALAXY,2.0,3,1.6234,0.000534,--|Possible [OII] next to sky lines| no features,3,none


In [87]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.6229
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. secure line, broadened but not resolved. no other secure features, flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
345,35185778208281832,-1,0.33.0.dev3890,0,GALAXY,1.6243,ACE,2.0,--,1.6230,...,91.156236,1.6229,GALAXY,2.0,3,0.000534,1.6234,--|Possible [OII] next to sky lines| no features,3,"disagreement in flag. secure line, broadened b..."
395,35185778208281832,-1,0.33.0.dev3890,0,GALAXY,1.6243,TWL,2.0,--,1.6229,...,91.156236,1.6229,GALAXY,2.0,3,0.000534,1.6234,--|Possible [OII] next to sky lines| no features,3,"disagreement in flag. secure line, broadened b..."
445,35185778208281832,-1,0.33.0.dev3890,0,GALAXY,1.6243,BJW,2.0,R,--,...,91.156236,1.6229,GALAXY,2.0,3,0.000534,1.6234,--|Possible [OII] next to sky lines| no features,3,"disagreement in flag. secure line, broadened b..."


conflict ID =  34


In [88]:
#copy this text to a new cell to display the conflict
conflict_id=35
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
346,35185778212471275,GALAXY,1.1584,ACE,4.0,RC,0.0000,STAR,--,0.0000,STAR,3.333333,2,1.1584,1.1584,--| Point source| could be a QSO with broad Mg...,3,none
396,35185778212471275,GALAXY,1.1584,TWL,4.0,RC,--,STAR,Point source,1.1584,STAR,3.333333,2,1.1584,1.1584,--| Point source| could be a QSO with broad Mg...,3,none
446,35185778212471275,GALAXY,1.1584,BJW,2.0,RC,1.1350,QSO,could be a QSO with broad Mg II if bump is real,1.1350,QSO,3.333333,2,1.1584,1.1584,--| Point source| could be a QSO with broad Mg...,3,none


In [89]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'STAR'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. STAR.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
346,35185778212471275,-1,0.33.0.dev3890,0,GALAXY,1.1584,ACE,4.0,RC,0.0000,...,6.314427,0.0,STAR,3.333333,2,1.1584,1.1584,--| Point source| could be a QSO with broad Mg...,3,disagreement in all. STAR.
396,35185778212471275,-1,0.33.0.dev3890,0,GALAXY,1.1584,TWL,4.0,RC,--,...,6.314427,0.0,STAR,3.333333,2,1.1584,1.1584,--| Point source| could be a QSO with broad Mg...,3,disagreement in all. STAR.
446,35185778212471275,-1,0.33.0.dev3890,0,GALAXY,1.1584,BJW,4.0,RC,1.1350,...,6.314427,0.0,STAR,3.333333,2,1.1584,1.1584,--| Point source| could be a QSO with broad Mg...,3,disagreement in all. STAR.


conflict ID =  35


In [90]:
#copy this text to a new cell to display the conflict
conflict_id=36
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
461,35185778212474885,GALAXY,0.9559,BJW,0.0,--,--,--,--,0.9559,GALAXY,1.333333,3,0.9559,0.7052,--| Some continuum| Very narrow Lyman alpha?,3,none
511,35185778212474885,GALAXY,0.9559,TWL,1.0,--,--,--,Some continuum,0.9559,GALAXY,1.333333,3,0.9559,0.7052,--| Some continuum| Very narrow Lyman alpha?,3,none
561,35185778212474885,GALAXY,0.9559,ACE,3.0,--,2.3352,QSO,Very narrow Lyman alpha?,2.3352,QSO,1.333333,3,0.9559,0.7052,--| Some continuum| Very narrow Lyman alpha?,3,none


In [91]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9559
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. too low SNR. Take RR answer, flag 1'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
461,35185778212474885,-1,0.33.0.dev3890,0,GALAXY,0.9559,BJW,1.0,--,--,...,4.90877,0.9559,GALAXY,1.333333,3,0.7052,0.9559,--| Some continuum| Very narrow Lyman alpha?,3,disagreement in all. too low SNR. Take RR answ...
511,35185778212474885,-1,0.33.0.dev3890,0,GALAXY,0.9559,TWL,1.0,--,--,...,4.90877,0.9559,GALAXY,1.333333,3,0.7052,0.9559,--| Some continuum| Very narrow Lyman alpha?,3,disagreement in all. too low SNR. Take RR answ...
561,35185778212474885,-1,0.33.0.dev3890,0,GALAXY,0.9559,ACE,1.0,--,2.3352,...,4.90877,0.9559,GALAXY,1.333333,3,0.7052,0.9559,--| Some continuum| Very narrow Lyman alpha?,3,disagreement in all. too low SNR. Take RR answ...


conflict ID =  36


In [92]:
#copy this text to a new cell to display the conflict
conflict_id=37
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
467,35185778216665338,GALAXY,0.4628,BJW,1.0,R,--,--,nominal line doesn't seem real,0.4628,GALAXY,2.333333,3,0.4628,0.343177,nominal line doesn't seem real| Not sure the ...,3,none
517,35185778216665338,GALAXY,0.4628,TWL,2.0,--,--,--,Not sure the model is a good fit,0.4628,GALAXY,2.333333,3,0.4628,0.343177,nominal line doesn't seem real| Not sure the ...,3,none
567,35185778216665338,GALAXY,0.4628,ACE,4.0,--,0.9648,GALAXY,MgII in absorption.,0.9648,GALAXY,2.333333,3,0.4628,0.343177,nominal line doesn't seem real| Not sure the ...,3,none


In [93]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9648
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. OII doublet missed by RR. take VI z with flag 3.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
467,35185778216665338,-1,0.33.0.dev3890,0,GALAXY,0.4628,BJW,3.0,R,--,...,4.84046,0.9648,GALAXY,2.333333,3,0.343177,0.4628,nominal line doesn't seem real| Not sure the ...,3,disagreement in z and flag. OII doublet missed...
517,35185778216665338,-1,0.33.0.dev3890,0,GALAXY,0.4628,TWL,3.0,--,--,...,4.84046,0.9648,GALAXY,2.333333,3,0.343177,0.4628,nominal line doesn't seem real| Not sure the ...,3,disagreement in z and flag. OII doublet missed...
567,35185778216665338,-1,0.33.0.dev3890,0,GALAXY,0.4628,ACE,3.0,--,0.9648,...,4.84046,0.9648,GALAXY,2.333333,3,0.343177,0.4628,nominal line doesn't seem real| Not sure the ...,3,disagreement in z and flag. OII doublet missed...


conflict ID =  37


In [94]:
#copy this text to a new cell to display the conflict
conflict_id=38
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
476,35185778216667033,GALAXY,0.987,BJW,0.0,R,--,--,--,0.9870,GALAXY,2.333333,4,0.987,0.985809,--| Point source|White dwarf.,3,none
526,35185778216667033,GALAXY,0.987,TWL,4.0,RC,--,STAR,Point source,0.9870,STAR,2.333333,4,0.987,0.985809,--| Point source|White dwarf.,3,none
576,35185778216667033,GALAXY,0.987,ACE,3.0,--,0.0006,STAR,White dwarf.,0.0006,STAR,2.333333,4,0.987,0.985809,--| Point source|White dwarf.,3,none


In [95]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.0006
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'STAR'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. STAR.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
476,35185778216667033,-1,0.33.0.dev3890,0,GALAXY,0.987,BJW,3.0,R,--,...,14.192729,0.0006,STAR,2.333333,4,0.985809,0.987,--| Point source|White dwarf.,3,disagreement in all. STAR.
526,35185778216667033,-1,0.33.0.dev3890,0,GALAXY,0.987,TWL,3.0,RC,--,...,14.192729,0.0006,STAR,2.333333,4,0.985809,0.987,--| Point source|White dwarf.,3,disagreement in all. STAR.
576,35185778216667033,-1,0.33.0.dev3890,0,GALAXY,0.987,ACE,3.0,--,0.0006,...,14.192729,0.0006,STAR,2.333333,4,0.985809,0.987,--| Point source|White dwarf.,3,disagreement in all. STAR.


conflict ID =  38


In [96]:
#copy this text to a new cell to display the conflict
conflict_id=39
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
478,35185778216667241,GALAXY,1.2219,BJW,3.0,--,--,--,--,1.2219,GALAXY,3.0,2,1.2219,0.0,--|Possible [OII] on sky lines|--,3,none
528,35185778216667241,GALAXY,1.2219,TWL,2.0,--,--,--,Possible [OII] on sky lines,1.2219,GALAXY,3.0,2,1.2219,0.0,--|Possible [OII] on sky lines|--,3,none
578,35185778216667241,GALAXY,1.2219,ACE,4.0,--,--,--,--,1.2219,GALAXY,3.0,2,1.2219,0.0,--|Possible [OII] on sky lines|--,3,none


In [97]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
478,35185778216667241,-1,0.33.0.dev3890,0,GALAXY,1.2219,BJW,2.0,--,--,...,48.251677,1.2219,GALAXY,3.0,2,0.0,1.2219,--|Possible [OII] on sky lines|--,3,flag disagreement. z not secure.
528,35185778216667241,-1,0.33.0.dev3890,0,GALAXY,1.2219,TWL,2.0,--,--,...,48.251677,1.2219,GALAXY,3.0,2,0.0,1.2219,--|Possible [OII] on sky lines|--,3,flag disagreement. z not secure.
578,35185778216667241,-1,0.33.0.dev3890,0,GALAXY,1.2219,ACE,2.0,--,--,...,48.251677,1.2219,GALAXY,3.0,2,0.0,1.2219,--|Possible [OII] on sky lines|--,3,flag disagreement. z not secure.


conflict ID =  39


In [98]:
#copy this text to a new cell to display the conflict
conflict_id=40
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
483,35185778216668354,GALAXY,1.0199,BJW,1.0,--,--,--,features weak or absent,1.0199,GALAXY,2.333333,2,1.0199,2.2508,features weak or absent| Point source| Strong...,3,none
533,35185778216668354,GALAXY,1.0199,TWL,3.0,RC,0.0000,STAR,Point source,0.0000,STAR,2.333333,2,1.0199,2.2508,features weak or absent| Point source| Strong...,3,none
583,35185778216668354,GALAXY,1.0199,ACE,3.0,RC,2.2508,QSO,Strong Lyman alpha only,2.2508,QSO,2.333333,2,1.0199,2.2508,features weak or absent| Point source| Strong...,3,none


In [99]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0199
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. no secure features. take RR fit with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
483,35185778216668354,-1,0.33.0.dev3890,0,GALAXY,1.0199,BJW,1.0,--,--,...,8.394417,1.0199,GALAXY,2.333333,2,2.2508,1.0199,features weak or absent| Point source| Strong...,3,disagreement in all. no secure features. take ...
533,35185778216668354,-1,0.33.0.dev3890,0,GALAXY,1.0199,TWL,1.0,RC,0.0000,...,8.394417,1.0199,GALAXY,2.333333,2,2.2508,1.0199,features weak or absent| Point source| Strong...,3,disagreement in all. no secure features. take ...
583,35185778216668354,-1,0.33.0.dev3890,0,GALAXY,1.0199,ACE,1.0,RC,2.2508,...,8.394417,1.0199,GALAXY,2.333333,2,2.2508,1.0199,features weak or absent| Point source| Strong...,3,disagreement in all. no secure features. take ...


conflict ID =  40


In [103]:
#copy this text to a new cell to display the conflict
conflict_id=41
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
490,35185778216669670,GALAXY,0.9874,BJW,0.0,RS,--,--,bad continuum - dip,0.9874,GALAXY,1.333333,3,0.9874,0.9874,bad continuum - dip| Point source| No clear ...,3,none
540,35185778216669670,GALAXY,0.9874,TWL,3.0,--,0.0000,STAR,Point source,0.0000,STAR,1.333333,3,0.9874,0.9874,bad continuum - dip| Point source| No clear ...,3,none
590,35185778216669670,GALAXY,0.9874,ACE,1.0,--,--,--,No clear match to features.,0.9874,GALAXY,1.333333,3,0.9874,0.9874,bad continuum - dip| Point source| No clear ...,3,none


In [104]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9874
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. no secure features. take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
490,35185778216669670,-1,0.33.0.dev3890,0,GALAXY,0.9874,BJW,1.0,RS,--,...,3.777937,0.9874,GALAXY,1.333333,3,0.9874,0.9874,bad continuum - dip| Point source| No clear ...,3,disagreement in all. no secure features. take ...
540,35185778216669670,-1,0.33.0.dev3890,0,GALAXY,0.9874,TWL,1.0,--,0.0000,...,3.777937,0.9874,GALAXY,1.333333,3,0.9874,0.9874,bad continuum - dip| Point source| No clear ...,3,disagreement in all. no secure features. take ...
590,35185778216669670,-1,0.33.0.dev3890,0,GALAXY,0.9874,ACE,1.0,--,--,...,3.777937,0.9874,GALAXY,1.333333,3,0.9874,0.9874,bad continuum - dip| Point source| No clear ...,3,disagreement in all. no secure features. take ...


conflict ID =  41


In [107]:
#copy this text to a new cell to display the conflict
conflict_id=42
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
604,35185778220861973,GALAXY,1.5498,HSH,0.0,R,--,--,--,1.5498,GALAXY,2.0,3,1.5498,0.0,--| [OII] emission next to sky lines|--,3,none
654,35185778220861973,GALAXY,1.5498,TWL,3.0,--,--,--,[OII] emission next to sky lines,1.5498,GALAXY,2.0,3,1.5498,0.0,--| [OII] emission next to sky lines|--,3,none
704,35185778220861973,GALAXY,1.5498,ACE,3.0,--,--,--,--,1.5498,GALAXY,2.0,3,1.5498,0.0,--| [OII] emission next to sky lines|--,3,none


In [108]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. OII compromised, no other features, z not secure. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
604,35185778220861973,-1,0.33.0.dev3890,0,GALAXY,1.5498,HSH,2.0,R,--,...,17.675252,1.5498,GALAXY,2.0,3,0.0,1.5498,--| [OII] emission next to sky lines|--,3,"flag disagreement, take mean. OII compromised,..."
654,35185778220861973,-1,0.33.0.dev3890,0,GALAXY,1.5498,TWL,2.0,--,--,...,17.675252,1.5498,GALAXY,2.0,3,0.0,1.5498,--| [OII] emission next to sky lines|--,3,"flag disagreement, take mean. OII compromised,..."
704,35185778220861973,-1,0.33.0.dev3890,0,GALAXY,1.5498,ACE,2.0,--,--,...,17.675252,1.5498,GALAXY,2.0,3,0.0,1.5498,--| [OII] emission next to sky lines|--,3,"flag disagreement, take mean. OII compromised,..."


conflict ID =  42


In [111]:
#copy this text to a new cell to display the conflict
conflict_id=43
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
611,35185778220863277,QSO,2.7687,HSH,2.0,--,--,--,I am not sure the QSO spectral type is correct.,2.7687,QSO,2.666667,2,2.7687,0.0,I am not sure the QSO spectral type is correct...,3,none
661,35185778220863277,QSO,2.7687,TWL,2.0,--,--,--,Low S/N; Hard to tell whether the broad emiss...,2.7687,QSO,2.666667,2,2.7687,0.0,I am not sure the QSO spectral type is correct...,3,none
711,35185778220863277,QSO,2.7687,ACE,4.0,--,--,--,--,2.7687,QSO,2.666667,2,2.7687,0.0,I am not sure the QSO spectral type is correct...,3,none


In [112]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 2.7687
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. no secure features, low signal and calibration issues. not secure. not a QSO.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
611,35185778220863277,-1,0.33.0.dev3890,0,QSO,2.7687,HSH,1.0,--,--,...,13.634759,2.7687,GALAXY,2.666667,2,0.0,2.7687,I am not sure the QSO spectral type is correct...,3,"disagreement in flag. no secure features, low ..."
661,35185778220863277,-1,0.33.0.dev3890,0,QSO,2.7687,TWL,1.0,--,--,...,13.634759,2.7687,GALAXY,2.666667,2,0.0,2.7687,I am not sure the QSO spectral type is correct...,3,"disagreement in flag. no secure features, low ..."
711,35185778220863277,-1,0.33.0.dev3890,0,QSO,2.7687,ACE,1.0,--,--,...,13.634759,2.7687,GALAXY,2.666667,2,0.0,2.7687,I am not sure the QSO spectral type is correct...,3,"disagreement in flag. no secure features, low ..."


conflict ID =  43


In [115]:
#copy this text to a new cell to display the conflict
conflict_id=44
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
616,35185778220864335,GALAXY,1.2542,HSH,2.0,--,--,--,--,1.2542,GALAXY,2.333333,3,1.2542,0.615961,--| Model fits sky residuals|--,3,none
666,35185778220864335,GALAXY,1.2542,TWL,1.0,--,--,--,Model fits sky residuals,1.2542,GALAXY,2.333333,3,1.2542,0.615961,--| Model fits sky residuals|--,3,none
716,35185778220864335,GALAXY,1.2542,ACE,4.0,R,2.6427,QSO,--,2.6427,QSO,2.333333,3,1.2542,0.615961,--| Model fits sky residuals|--,3,none


In [116]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2542
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. No secure features. Take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
616,35185778220864335,-1,0.33.0.dev3890,0,GALAXY,1.2542,HSH,1.0,--,--,...,9.743423,1.2542,GALAXY,2.333333,3,0.615961,1.2542,--| Model fits sky residuals|--,3,disagreement in all. No secure features. Take ...
666,35185778220864335,-1,0.33.0.dev3890,0,GALAXY,1.2542,TWL,1.0,--,--,...,9.743423,1.2542,GALAXY,2.333333,3,0.615961,1.2542,--| Model fits sky residuals|--,3,disagreement in all. No secure features. Take ...
716,35185778220864335,-1,0.33.0.dev3890,0,GALAXY,1.2542,ACE,1.0,R,2.6427,...,9.743423,1.2542,GALAXY,2.333333,3,0.615961,1.2542,--| Model fits sky residuals|--,3,disagreement in all. No secure features. Take ...


conflict ID =  44


In [119]:
#copy this text to a new cell to display the conflict
conflict_id=45
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
617,35185778220864810,GALAXY,0.4128,HSH,2.0,R,--,--,--,0.4128,GALAXY,3.0,2,0.4128,0.760617,--| Bad calibration drives the bestfit| Clear OII,3,none
667,35185778220864810,GALAXY,0.4128,TWL,4.0,R,1.4874,--,Bad calibration drives the bestfit,1.4874,GALAXY,3.0,2,0.4128,0.760617,--| Bad calibration drives the bestfit| Clear OII,3,none
717,35185778220864810,GALAXY,0.4128,ACE,3.0,R,1.4871,GALAXY,Clear OII,1.4871,GALAXY,3.0,2,0.4128,0.760617,--| Bad calibration drives the bestfit| Clear OII,3,none


In [120]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.4871
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'broadened but unresolved line. RR says Ha, 2/3 VI say OII. Potential change in continuum consistent with OII. Take VI z, flag 2.5. '

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
617,35185778220864810,-1,0.33.0.dev3890,0,GALAXY,0.4128,HSH,2.5,R,--,...,5.194871,1.4871,GALAXY,3.0,2,0.760617,0.4128,--| Bad calibration drives the bestfit| Clear OII,3,"broadened but unresolved line. RR says Ha, 2/3..."
667,35185778220864810,-1,0.33.0.dev3890,0,GALAXY,0.4128,TWL,2.5,R,1.4874,...,5.194871,1.4871,GALAXY,3.0,2,0.760617,0.4128,--| Bad calibration drives the bestfit| Clear OII,3,"broadened but unresolved line. RR says Ha, 2/3..."
717,35185778220864810,-1,0.33.0.dev3890,0,GALAXY,0.4128,ACE,2.5,R,1.4871,...,5.194871,1.4871,GALAXY,3.0,2,0.760617,0.4128,--| Bad calibration drives the bestfit| Clear OII,3,"broadened but unresolved line. RR says Ha, 2/3..."


conflict ID =  45


In [123]:
#copy this text to a new cell to display the conflict
conflict_id=46
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
618,35185778225053791,QSO,1.5109,HSH,2.0,R,--,--,--,1.5109,QSO,3.0,2,1.511167,0.000159,--| [OII] emission between skylines|--,3,none
668,35185778225053791,QSO,1.5109,TWL,4.0,--,1.5113,--,[OII] emission between skylines,1.5113,QSO,3.0,2,1.511167,0.000159,--| [OII] emission between skylines|--,3,none
718,35185778225053791,QSO,1.5109,ACE,3.0,--,1.5113,GALAXY,--,1.5113,GALAXY,3.0,2,1.511167,0.000159,--| [OII] emission between skylines|--,3,none


In [124]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.5113
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. not a QSO. slight shift to better match lines.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
618,35185778225053791,-1,0.33.0.dev3890,0,QSO,1.5109,HSH,3.0,R,--,...,46.828061,1.5113,GALAXY,3.0,2,0.000159,1.511167,--| [OII] emission between skylines|--,3,"flag disagreement, take mean. not a QSO. sligh..."
668,35185778225053791,-1,0.33.0.dev3890,0,QSO,1.5109,TWL,3.0,--,1.5113,...,46.828061,1.5113,GALAXY,3.0,2,0.000159,1.511167,--| [OII] emission between skylines|--,3,"flag disagreement, take mean. not a QSO. sligh..."
718,35185778225053791,-1,0.33.0.dev3890,0,QSO,1.5109,ACE,3.0,--,1.5113,...,46.828061,1.5113,GALAXY,3.0,2,0.000159,1.511167,--| [OII] emission between skylines|--,3,"flag disagreement, take mean. not a QSO. sligh..."


conflict ID =  46


In [127]:
#copy this text to a new cell to display the conflict
conflict_id=47
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
619,35185778225053999,GALAXY,1.0004,HSH,2.0,--,--,--,--,1.0004,GALAXY,3.333333,2,1.0004,0.0,--|--|--,3,none
669,35185778225053999,GALAXY,1.0004,TWL,4.0,--,--,--,--,1.0004,GALAXY,3.333333,2,1.0004,0.0,--|--|--,3,none
719,35185778225053999,GALAXY,1.0004,ACE,4.0,--,--,--,--,1.0004,GALAXY,3.333333,2,1.0004,0.0,--|--|--,3,none


In [128]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
619,35185778225053999,-1,0.33.0.dev3890,0,GALAXY,1.0004,HSH,3.333333,--,--,...,315.50399,1.0004,GALAXY,3.333333,2,0.0,1.0004,--|--|--,3,"flag disagreement, take mean."
669,35185778225053999,-1,0.33.0.dev3890,0,GALAXY,1.0004,TWL,3.333333,--,--,...,315.50399,1.0004,GALAXY,3.333333,2,0.0,1.0004,--|--|--,3,"flag disagreement, take mean."
719,35185778225053999,-1,0.33.0.dev3890,0,GALAXY,1.0004,ACE,3.333333,--,--,...,315.50399,1.0004,GALAXY,3.333333,2,0.0,1.0004,--|--|--,3,"flag disagreement, take mean."


conflict ID =  47


In [131]:
#copy this text to a new cell to display the conflict
conflict_id=48
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
622,35185778225054325,GALAXY,0.4918,HSH,1.0,R,--,--,--,0.4918,GALAXY,2.0,3,0.4918,2.4823,--| Some continuum; could be a star|--,3,none
672,35185778225054325,GALAXY,0.4918,TWL,1.0,RC,0,STAR,Some continuum; could be a star,0.0000,STAR,2.0,3,0.4918,2.4823,--| Some continuum; could be a star|--,3,none
722,35185778225054325,GALAXY,0.4918,ACE,4.0,RC,2.4823,QSO,--,2.4823,QSO,2.0,3,0.4918,2.4823,--| Some continuum; could be a star|--,3,none


In [132]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.4918
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. no secure features. calibration issues. take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
622,35185778225054325,-1,0.33.0.dev3890,0,GALAXY,0.4918,HSH,1.0,R,--,...,3.211636,0.4918,GALAXY,2.0,3,2.4823,0.4918,--| Some continuum; could be a star|--,3,disagreement in all. no secure features. calib...
672,35185778225054325,-1,0.33.0.dev3890,0,GALAXY,0.4918,TWL,1.0,RC,0,...,3.211636,0.4918,GALAXY,2.0,3,2.4823,0.4918,--| Some continuum; could be a star|--,3,disagreement in all. no secure features. calib...
722,35185778225054325,-1,0.33.0.dev3890,0,GALAXY,0.4918,ACE,1.0,RC,2.4823,...,3.211636,0.4918,GALAXY,2.0,3,2.4823,0.4918,--| Some continuum; could be a star|--,3,disagreement in all. no secure features. calib...


conflict ID =  48


In [135]:
#copy this text to a new cell to display the conflict
conflict_id=49
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
624,35185778225054792,GALAXY,1.2021,HSH,2.0,--,--,--,--,1.2021,GALAXY,3.333333,2,1.2021,0.0,--|--|--,3,none
674,35185778225054792,GALAXY,1.2021,TWL,4.0,--,--,--,--,1.2021,GALAXY,3.333333,2,1.2021,0.0,--|--|--,3,none
724,35185778225054792,GALAXY,1.2021,ACE,4.0,--,--,--,--,1.2021,GALAXY,3.333333,2,1.2021,0.0,--|--|--,3,none


In [136]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
624,35185778225054792,-1,0.33.0.dev3890,0,GALAXY,1.2021,HSH,3.333333,--,--,...,297.006001,1.2021,GALAXY,3.333333,2,0.0,1.2021,--|--|--,3,"flag disagreement, take mean."
674,35185778225054792,-1,0.33.0.dev3890,0,GALAXY,1.2021,TWL,3.333333,--,--,...,297.006001,1.2021,GALAXY,3.333333,2,0.0,1.2021,--|--|--,3,"flag disagreement, take mean."
724,35185778225054792,-1,0.33.0.dev3890,0,GALAXY,1.2021,ACE,3.333333,--,--,...,297.006001,1.2021,GALAXY,3.333333,2,0.0,1.2021,--|--|--,3,"flag disagreement, take mean."


conflict ID =  49


In [140]:
#copy this text to a new cell to display the conflict
conflict_id=50
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
628,35185778225055676,GALAXY,0.9038,HSH,2.0,--,--,--,--,0.9038,GALAXY,2.666667,2,0.9038,0.0,--| Weak emission lines; could be noise|--,3,none
678,35185778225055676,GALAXY,0.9038,TWL,2.0,--,--,--,Weak emission lines; could be noise,0.9038,GALAXY,2.666667,2,0.9038,0.0,--| Weak emission lines; could be noise|--,3,none
728,35185778225055676,GALAXY,0.9038,ACE,4.0,--,--,--,--,0.9038,GALAXY,2.666667,2,0.9038,0.0,--| Weak emission lines; could be noise|--,3,none


In [141]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. z not secure. '

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
628,35185778225055676,-1,0.33.0.dev3890,0,GALAXY,0.9038,HSH,2.0,--,--,...,14.320457,0.9038,GALAXY,2.666667,2,0.0,0.9038,--| Weak emission lines; could be noise|--,3,disagreement in flag. z not secure.
678,35185778225055676,-1,0.33.0.dev3890,0,GALAXY,0.9038,TWL,2.0,--,--,...,14.320457,0.9038,GALAXY,2.666667,2,0.0,0.9038,--| Weak emission lines; could be noise|--,3,disagreement in flag. z not secure.
728,35185778225055676,-1,0.33.0.dev3890,0,GALAXY,0.9038,ACE,2.0,--,--,...,14.320457,0.9038,GALAXY,2.666667,2,0.0,0.9038,--| Weak emission lines; could be noise|--,3,disagreement in flag. z not secure.


conflict ID =  50


In [144]:
#copy this text to a new cell to display the conflict
conflict_id=51
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
638,35185778225057501,GALAXY,1.278,HSH,2.0,--,--,--,--,1.278,GALAXY,3.333333,2,1.278,0.0,--|--|--,3,none
688,35185778225057501,GALAXY,1.278,TWL,4.0,--,--,--,--,1.278,GALAXY,3.333333,2,1.278,0.0,--|--|--,3,none
738,35185778225057501,GALAXY,1.278,ACE,4.0,--,--,--,--,1.278,GALAXY,3.333333,2,1.278,0.0,--|--|--,3,none


In [145]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
638,35185778225057501,-1,0.33.0.dev3890,0,GALAXY,1.278,HSH,3.333333,--,--,...,94.729852,1.278,GALAXY,3.333333,2,0.0,1.278,--|--|--,3,"flag disagreement, take mean."
688,35185778225057501,-1,0.33.0.dev3890,0,GALAXY,1.278,TWL,3.333333,--,--,...,94.729852,1.278,GALAXY,3.333333,2,0.0,1.278,--|--|--,3,"flag disagreement, take mean."
738,35185778225057501,-1,0.33.0.dev3890,0,GALAXY,1.278,ACE,3.333333,--,--,...,94.729852,1.278,GALAXY,3.333333,2,0.0,1.278,--|--|--,3,"flag disagreement, take mean."


conflict ID =  51


In [148]:
#copy this text to a new cell to display the conflict
conflict_id=52
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
648,35185778229252179,GALAXY,1.5739,HSH,0.0,R,--,--,--,1.5739,GALAXY,2.333333,4,1.5739,0.0,--|Broad [OII] emission|--,3,none
698,35185778229252179,GALAXY,1.5739,TWL,4.0,--,--,--,Broad [OII] emission,1.5739,GALAXY,2.333333,4,1.5739,0.0,--|Broad [OII] emission|--,3,none
748,35185778229252179,GALAXY,1.5739,ACE,3.0,--,--,--,--,1.5739,GALAXY,2.333333,4,1.5739,0.0,--|Broad [OII] emission|--,3,none


In [149]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. borderline resolved doublet. take as secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
648,35185778229252179,-1,0.33.0.dev3890,0,GALAXY,1.5739,HSH,3.0,R,--,...,127.203264,1.5739,GALAXY,2.333333,4,0.0,1.5739,--|Broad [OII] emission|--,3,disagreement in flag. borderline resolved doub...
698,35185778229252179,-1,0.33.0.dev3890,0,GALAXY,1.5739,TWL,3.0,--,--,...,127.203264,1.5739,GALAXY,2.333333,4,0.0,1.5739,--|Broad [OII] emission|--,3,disagreement in flag. borderline resolved doub...
748,35185778229252179,-1,0.33.0.dev3890,0,GALAXY,1.5739,ACE,3.0,--,--,...,127.203264,1.5739,GALAXY,2.333333,4,0.0,1.5739,--|Broad [OII] emission|--,3,disagreement in flag. borderline resolved doub...


conflict ID =  52


### and so on...

We should now recompute the conflicts, and not find any.

In [151]:
#vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
#                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
#                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
#                       & (len(x) >= 2)) #x is a group by TargetID
vi_conflict = VI.find_conflicts(vi_gp)

In [152]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


Display anything that was missed (if "Total number of conflicts" isn't zero) and resolve!

## Woohoo!

## Finally
we need to recompute vi_combined_flag and vi_combined_z, after everything is resolved.

In [153]:
vi['vi_combined_flag'] = vi.groupby('TargetID')['VI class'].transform('mean')
vi['vi_combined_z'] = vi.groupby('TargetID')['best redshift'].transform('mean')


## Now we prepare to write to file. 

### The important columns for the truth table construction are **vi_combined_flag** and **vi_combined_z** 

The truth table should **take the redhift value in vi_combined_z** for all objects with **vi_combined_flag >= 2.5** (for discussion, but this catches the cases where there is some disagreement on the 2/3 boundary).

After the merging process, those two columns will be the same for ALL of the members of each group by TargetID, so it doesn't matter which member we write to file. Taking the 1st one just makes this easy. 


In [154]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff', 'dz',
       'vi_combined_z', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')

In [157]:
print(VI_dir+save_filename)

/global/homes/r/rtojeiro/SV/VI_files/SV0/ELG/truth_table_ELGs_sets31_35.csv


In [158]:
#vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'all VI comments', 'merger comment', 'N_VI', 'DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+'merged_VI.txt')
vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments', 'merger comment','DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+save_filename)

Check that merged file reads in OK - check comments

In [159]:
merged_file = pd.read_csv(VI_dir+save_filename)

In [160]:
merged_file.keys()

Index(['TargetID', 'Redrock z', 'Redrock spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments',
       'merger comment', 'DELTACHI2', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z',
       'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z'],
      dtype='object')

In [162]:
merged_file

,TargetID,Redrock z,Redrock spectype,best redshift,best spectype,vi_combined_flag,N_VI,all VI comments,merger comment,DELTACHI2,FIBER,FLUX_G,FLUX_R,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z
0,35185778187306043,1.0142,GALAXY,1.0142,GALAXY,4.000000,3,--| OII and balmer lines|--,none,50.728473,1568,0.381515,0.749483,1.956381,0.174664,0.343126,0.895666
1,35185778187306542,0.9864,GALAXY,0.9864,GALAXY,4.000000,3,MgII in absorption.| OII and balmer lines|--,none,679.765608,1581,0.405773,0.440596,0.839144,0.277742,0.301578,0.574375
2,35185778187306622,0.6365,GALAXY,0.6365,GALAXY,4.000000,3,--| OII and balmer lines|--,none,710.741031,1573,0.514479,0.899860,2.462936,0.277349,0.485103,1.327738
3,35185778187306634,1.5635,GALAXY,1.5635,GALAXY,1.666667,3,OII on sky line.|--| Possible [OII] next to s...,none,20.478735,1803,0.326281,0.332590,0.835230,0.189153,0.192810,0.484203
4,35185778187306802,0.7823,GALAXY,0.7823,GALAXY,3.333333,3,--| OII| Broad [OII] emission,"flag disagreement, take mean.",344.300610,1591,0.522217,0.999329,2.898052,0.146363,0.280083,0.812241
5,35185778187307161,0.9542,GALAXY,0.9542,GALAXY,4.000000,3,--| OII doublet and weak balmer lines|--,none,132.605144,1814,0.356483,0.569721,1.682992,0.170592,0.272635,0.805382
6,35185778187307173,1.1923,GALAXY,1.1929,GALAXY,3.333333,3,--| OII doublet| Triple-peak [OII] emission,none,431.812797,1564,0.488851,0.664792,1.400834,0.313189,0.425907,0.897461
7,35185778187307360,1.5153,GALAXY,1.6162,GALAXY,2.000000,3,Weak OII| OII possible doublet| Model fits sk...,"flag disagreement, take mean. z not secure. po...",2.014799,1815,0.367955,0.404414,0.389181,0.286539,0.314931,0.303069
8,35185778187307415,1.3840,GALAXY,1.3840,GALAXY,1.333333,3,Based on OII only| OII ?| Low S/N,none,8.023933,1585,0.336390,0.444708,0.789630,0.142185,0.187970,0.333761
9,35185778187307571,1.3320,GALAXY,1.3320,GALAXY,1.000000,3,Narrow Lyman alpha.| Hb| Not physical model,"disagreement in all. spectra too noisy, not se...",25.703903,1556,0.278531,0.259282,0.307779,0.194338,0.180907,0.214745


In [163]:
merged_file.loc[9]['merger comment']

'disagreement in all. spectra too noisy, not secure. take RR answer, flag 1.'

In [164]:
merged_file.loc[9]['all VI comments']

' Narrow Lyman alpha.| Hb| Not physical model'

In [165]:
VI_dir

'/global/homes/r/rtojeiro/SV/VI_files/SV0/ELG/'